In [3]:
import pandas as pd
import numpy as np
import math
import datetime
from datetime import timedelta

* 获取测试数据

In [2]:
base_path = '/home/yuchuchu/machine/machine_algo/用户管理中心/2流量生成模块/'
test_file = base_path + 'test0514.csv'
test_data = pd.read_csv(test_file)

In [ ]:
def get_plan_pra(plan_level):
    """返回不同档位的参数值

    Args:
        plan_level: 方案档位, low, medium, high, max

    Returns:
        _type_: 返回月度clv阈值(month_clv_threshold), 
                返回年度clv阈值year_clv_threshold, 
                返回渠道放大系数channel_max_adjust, 
                返回渠道偏好系数channel_preference
    """
    # 月度clv阈值
    # 年度clv等级
    # 渠道上限调节, 999表示取上限，1.5表示均值的1.5倍，1.2表示均值的1.2倍，等等
    # 渠道倾向 channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
    
    # plan_pra_dict = {'max' : [50, '1', 999, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'high' : [50, '1', 1.5, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'medium' : [50, '1', 1.2, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'low' : [50, '1', 1.0, [1.0, 1.0, 1.0, 1.0,1.0]]}
    
    if plan_level == 'max':
        month_clv_threshold = 50  
        year_clv_threshold = '1'  
        channel_max_adjust = 999  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    elif plan_level == 'high':
        month_clv_threshold = 50
        year_clv_threshold = '1'  
        channel_max_adjust = 1.5 
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    elif plan_level == 'medium':
        month_clv_threshold = 50  
        year_clv_threshold = '1'  
        channel_max_adjust = 1.2  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    else:
        plan_level = 'lowv2'
        month_clv_threshold = 50  
        year_clv_threshold = '1' 
        channel_max_adjust = 1.0  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  

    return month_clv_threshold, year_clv_threshold, channel_max_adjust, channel_preference

1.测试保量函数  
* 测试样例测试保量函数

In [23]:

def BasicGuarantee(data):
    com_login_cnt_last_month = data['com_login_cnt_last_month']
    free_login_cnt_last_month = data['free_login_cnt_last_month']
    strong_ctrl_login_cnt_last_month =  data['strong_ctrl_login_cnt_last_month']
    normal_ctrl_login_cnt_last_month = data['normal_ctrl_login_cnt_last_month']
    no_ctrl_login_cnt_last_month = data['no_ctrl_login_cnt_last_month']

    # 新增 #
    com_login_cnt_last_month = com_login_cnt_last_month if com_login_cnt_last_month>=0 else 0
    free_login_cnt_last_month = free_login_cnt_last_month if free_login_cnt_last_month>=0 else 0
    strong_ctrl_login_cnt_last_month = strong_ctrl_login_cnt_last_month if strong_ctrl_login_cnt_last_month>=0 else 0
    normal_ctrl_login_cnt_last_month = normal_ctrl_login_cnt_last_month if normal_ctrl_login_cnt_last_month>=0 else 0
    no_ctrl_login_cnt_last_month = no_ctrl_login_cnt_last_month if no_ctrl_login_cnt_last_month>=0 else 0
    # 新增 #
    
    channel_guaranteed = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    
    channel_guaranteed['com'] = max(math.ceil(com_login_cnt_last_month),0)
    channel_guaranteed['free'] = max(math.ceil(free_login_cnt_last_month),0)
    channel_guaranteed['strong_ctrl'] = max(math.ceil(strong_ctrl_login_cnt_last_month),0)
    channel_guaranteed['normal_ctrl'] = max(math.ceil(normal_ctrl_login_cnt_last_month),0)
    channel_guaranteed['no_ctrl'] = max(math.ceil(no_ctrl_login_cnt_last_month),0)
    
    return channel_guaranteed

In [17]:
df = {'com_login_cnt_last_month': [1.2, 2],
      'free_login_cnt_last_month': [3, 4],
      'strong_ctrl_login_cnt_last_month': [1, None],
      'normal_ctrl_login_cnt_last_month': [3, 4],
      'no_ctrl_login_cnt_last_month': [3, 4]}
test_data = pd.DataFrame(df)
test_data.head(2)

,com_login_cnt_last_month,free_login_cnt_last_month,strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month
0,1.2,3,1.0,3,3
1,2.0,4,NaN,4,4


In [24]:
test_data['basic_result'] = test_data.apply(BasicGuarantee, 
                                            axis = 1, #逐行执行BasicGuarantee函数
                                            )

list(test_data['basic_result'])

[{'com': 2, 'free': 3, 'strong_ctrl': 1, 'normal_ctrl': 3, 'no_ctrl': 3},
 {'com': 2, 'free': 4, 'strong_ctrl': 0, 'normal_ctrl': 4, 'no_ctrl': 4}]

* BasicGuarantee保量函数，没有处理异常值的能力，比如nan、9999以及负数，需要增加；

2.测试业务补量函数

In [25]:
def BusinessRuleSupplement(basic_result, user_label,  year_clv_threshold, gmv_year, clv, month_clv_threshold, 
                           com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                           normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month, 
                           my_hash_code,month_hash):
    """根据用户逻辑进行方案调整

    """
    channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
    businessrule_no_allocation_channel = []
    businessrule_plan = basic_result
    channel_businessrule_max = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    channel_businessrule_max_add = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    
   
    year_clv_threshold = 600.0

    if user_label == 'A1':
        # 根据业务逻辑调整渠道上限
        com_adjust_pra = 1.2
        hash_precent = 28
        msg_add = 1
    
        # 京东可控置0 
        businessrule_no_allocation_channel.append('normal_ctrl')

        # 商业化渠道补量
        # A1人群的商业化补量不超过20%
        # 商业化在上月基础上调整1.2倍的抽样分配结果
        com_adjust_day = sampling_supplement(channel_days = com_login_cnt_last_month,
                                             adjust_day = com_adjust_pra * com_login_cnt_last_month, # 1.2
                                             my_hash_code = my_hash_code)
        
        channel_businessrule_max['com'] = com_adjust_day  # A1人群的商业化补量不超过20%
   
        # 免费渠道补量
        businessrule_plan['free'] +=  1 
        channel_businessrule_max_add['free'] +=1
        
        # 短信渠道补量
        if (my_hash_code <= hash_precent):
            businessrule_plan['strong_ctrl'] += msg_add
            channel_businessrule_max_add['strong_ctrl'] += msg_add     
        else:
            pass
       
    elif user_label == 'A2':
        if clv <= month_clv_threshold:
            businessrule_plan = basic_result
            
            # clv<50, 如果京东可控保量=0 则按月维度抽样（50%+1）
            if businessrule_plan['normal_ctrl'] == 0  and month_hash < 50:
                businessrule_plan['normal_ctrl'] +=1
                channel_businessrule_max_add['normal_ctrl'] += 1
        else:
            pass

    elif user_label == 'B':
        # 返回基础方案 所有渠道不分配
        businessrule_plan =  basic_result 
        businessrule_no_allocation_channel += channel_type      

    elif user_label == 'C':
        if gmv_year <= year_clv_threshold:
            # 返回基础方案 所有渠道不分配
            businessrule_plan = basic_result
            businessrule_no_allocation_channel += channel_type 
        else:
            businessrule_plan['normal_ctrl'] = min(20, 1.5 * normal_ctrl_login_cnt_last_month, businessrule_plan['normal_ctrl'])
    
    return businessrule_plan, businessrule_no_allocation_channel, channel_businessrule_max, channel_businessrule_max_add

In [2]:
test_data.columns

NameError: name 'test_data' is not defined

In [28]:

day1_ago_runday = datetime.datetime.strftime(datetime.datetime.strptime(run_day,'%Y-%M-%d') - timedelta(days=1),'%Y-%M-%d')
day1_ago_runday

'2022-05-14'

In [30]:
# 测试5月选择的分配方式是否合理



#run_day = datetime.datetime.strptime(run_day,'%Y-%M-%d')
for num in range(17):
    run_day = datetime.datetime.strftime(first_allocation_day+ timedelta(days=num),'%Y-%M-%d')
    print(run_day)
    delta_day = (datetime.datetime.strptime(run_day, '%Y-%M-%d')-first_allocation_day).days
    print(delta_day)
    
    if delta_day == 0:
        #  首日分配
        print(">"*10,"进行首日分配") 

    elif delta_day%3 == 0:
        # 3日-高活 渠道内调整 执行首日分配算法
        print(">"*10,"进行高活用户 渠道内调整") 


    elif (delta_day==7) or ( (delta_day > 7) & ((delta_day-7)%6==0)):
        # 7日 监控期，然后高活6d周期 渠道间调整
        print(">"*10,"进行高活用户 渠道间调整") 


    elif delta_day%3 > 0:
        # 非首日 非调控期 取昨日分区减去昨日已完成
        print(">"*10,"非首日，非调控期，取昨日分区减去昨日已完成") 

2022-05-15
0
>>>>>>>>>> 进行首日分配
2022-05-16
1
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-17
2
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-18
3
>>>>>>>>>> 进行高活用户 渠道内调整
2022-05-19
4
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-20
5
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-21
6
>>>>>>>>>> 进行高活用户 渠道内调整
2022-05-22
7
>>>>>>>>>> 进行高活用户 渠道间调整
2022-05-23
8
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-24
9
>>>>>>>>>> 进行高活用户 渠道内调整
2022-05-25
10
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-26
11
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-27
12
>>>>>>>>>> 进行高活用户 渠道内调整
2022-05-28
13
>>>>>>>>>> 进行高活用户 渠道间调整
2022-05-29
14
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成
2022-05-30
15
>>>>>>>>>> 进行高活用户 渠道内调整
2022-05-31
16
>>>>>>>>>> 非首日，非调控期，取昨日分区减去昨日已完成


* 测试渠道间调整函数

In [5]:
import math
this_month_shengu_day = 4
between_channel_adjust = 1

channel_max_login_days = {'com' :10,
                              'free':10,
                              'strong_ctrl':10,
                              'normal_ctrl':10,
                              'no_ctrl':10}

channel_login_cnt_mtd = {'com' :5,
                            'free':9,
                            'strong_ctrl':9,
                            'normal_ctrl':1,
                            'no_ctrl':2}


if between_channel_adjust==1:
    
    
    for adjust_channel in channel_max_login_days:
        print(adjust_channel)
        if channel_max_login_days[adjust_channel] - channel_login_cnt_mtd[adjust_channel]<=2:
            print("up")
            channel_max_login_days[adjust_channel] = max(math.ceil(channel_max_login_days[adjust_channel] * 1.1),channel_login_cnt_mtd[adjust_channel]+2)
        
        elif channel_max_login_days[adjust_channel] - channel_login_cnt_mtd[adjust_channel] >= this_month_shengu_day*1.5 :
            print("down")
            channel_max_login_days[adjust_channel] = max(math.ceil(channel_max_login_days[adjust_channel] * 0.9),math.ceil(this_month_shengu_day*1.5))

com
free
up
strong_ctrl
up
normal_ctrl
down
no_ctrl
down


In [3]:
channel_max_login_days

{'com': 10, 'free': 11, 'strong_ctrl': 11, 'normal_ctrl': 9, 'no_ctrl': 9}

In [1]:
# date: 2022-04-20
# updta: 2022-05-10
# author: niujianxing,yuchuchu

"""
用户管理中心-算法输出中间表
"""
import time
import datetime
from datetime import timedelta
import math
import sys

# old_time放在程序运行开始的地方
old_time = time.time()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, MapType
from pyspark.sql.functions import col, lit, array
from pyspark.sql.functions import udf

# import os
# os.environ['PYSPARK_PYTHON'] = "/usr/local/anaconda3/bin/python3.6"


spark = (SparkSession \
    .builder \
    .appName("test-dockerlinuxcontainer") \
    .enableHiveSupport() \
    .config("spark.sql.shuffle.partitions", "1000") \
    .getOrCreate())

spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")
spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "true")
spark.sql("""ADD JAR hdfs://ns1009/user/mart_jypt/mart_jypt_usr_grow/liuyang266/hiveudf-1.0-SNAPSHOT-jar-with-dependencies.jar""")
spark.sql("""CREATE TEMPORARY FUNCTION hash_sub AS 'com.jd.bdptools.ly.HashMonthly'""")

DataFrame[]

In [2]:
def get_base_data(part_dt,part_month):
    """获取sql文件

    Args:
        part_dt (_type_): 表格分区

    Returns:
        _type_: 返回执行sql
    """
    used_sql = """
    SELECT
        user.dt,
        user.user_log_acct,
        user.my_hash_code,
        user.month_hash,
        --格子粒度
        user.user_life_cycle_type_1st,
        user.model_a_1st,
        user.model_b_1st,
        user.goal_group_1st,
        user.annual_clv_1st,
        user.grid_name_1st,
        user.area_b_1st,

        --格子粒度2
        user.user_life_cycle_type,
        user.model_a,
        user.model_b,
        user.model_c,
        user.model_l,
        user.priority_type,
        
        user.is_malice_user,
        
        -- 目标登录次数
        COALESCE(grid.lower_lift_login_cnt,0) as lower_lift_login_cnt,
        COALESCE(user.natural_pred_login_cnt_rest,0) as natural_pred_login_cnt_rest,
        COALESCE(user.natural_pred_login_cnt_n30d,0) as natural_pred_login_cnt_n30d,
        
        -- 自然相关
        COALESCE(user.natural_login_cnt_last_month,0) as natural_login_cnt_last_month,
        COALESCE(grid.natural_max_login_cnt,0) as natural_max_login_cnt,
        COALESCE(grid.natural_login_cnt_per_month,0) as natural_login_cnt_per_month,
        COALESCE(grid.natural_quality_coeff,0) as natural_quality_coeff,
        COALESCE(grid.natural_login_cnt_per_dau,0) as natural_login_cnt_per_dau,

        --当月引流登端次数
        COALESCE(user.login_cnt_mtd,0) as login_cnt_mtd,
        user.natural_login_cnt_mtd,
        user.com_login_cnt_mtd,
        user.free_login_cnt_mtd,
        user.strong_ctrl_login_cnt_mtd,
        user.normal_ctrl_login_cnt_mtd,
        user.no_ctrl_login_cnt_mtd,

        --渠道是否可达
        user.is_acce_by_com,
        user.is_acce_by_free,
        user.is_acce_by_strong_ctrl,
        user.is_acce_by_normal_ctrl,
        user.is_acce_by_no_ctrl,

        --clv
        user.clv_pred_1m,
        user.clv_mtd,

        --上月人均干预次数
        COALESCE(login_cnt_last_month,0)         as login_cnt_last_month,
        COALESCE(com_login_cnt_last_month,0) as com_login_cnt_last_month,
        COALESCE(free_login_cnt_last_month,0) as free_login_cnt_last_month,
        COALESCE(strong_ctrl_login_cnt_last_month,0) as strong_ctrl_login_cnt_last_month,
        COALESCE(normal_ctrl_login_cnt_last_month,0) as normal_ctrl_login_cnt_last_month,
        COALESCE(no_ctrl_login_cnt_last_month,0) as no_ctrl_login_cnt_last_month,
        
        --gmv
        cast(COALESCE(gmv_year,0.0) as double)  as gmv_year,
        cast(COALESCE(last_year_gmv,0.0) as double) as last_year_gmv,
                
        --渠道上限
        COALESCE(grid.com_max_login_count,2) as com_max_login_count,
        COALESCE(grid.free_max_login_count,2) as free_max_login_count,
        COALESCE(grid.strong_ctrl_max_login_count,0) as strong_ctrl_max_login_count,
        COALESCE(grid.normal_ctrl_max_login_count,0) as normal_ctrl_max_login_count,
        COALESCE(grid.no_ctrl_max_login_count,0) as no_ctrl_max_login_count,

        --渠道成本
        COALESCE(grid.com_login_cost,-3) as com_login_cost,
        COALESCE(grid.free_login_cost,0) as free_login_cost,
        COALESCE(grid.no_ctrl_login_cost,0.9) as no_ctrl_login_cost,
        COALESCE(grid.strong_ctrl_login_cost,0.8) as strong_ctrl_login_cost,
        COALESCE(grid.normal_login_cost,0.6) as normal_login_cost,

        --渠道成本质量因子
        com_quality_coeff,
        free_quality_coeff,
        strong_ctrl_quality_coeff,
        normal_ctrl_quality_coeff,
        no_ctrl_quality_coeff,


        --渠道dau映射
        COALESCE(grid.com_login_cnt_per_dau,0.5)            as com_login_cnt_per_dau,
        COALESCE(grid.free_login_cnt_per_dau,0.5)           as free_login_cnt_per_dau,
        COALESCE(grid.no_ctrl_login_cnt_per_dau,0.5)        as no_ctrl_login_cnt_per_dau,
        COALESCE(grid.strong_ctrl_login_cnt_per_dau,0.5)    as strong_ctrl_login_cnt_per_dau,
        COALESCE(grid.normal_ctrl_login_cnt_per_dau,0.5)         as normal_ctrl_login_cnt_per_dau,
        grid.total_login_dau_ratio,

        -- 格子用户数& 渠道月均引流 for 保量
        case when grid.user_cnt <-100 then 1 else grid.user_cnt end as user_cnt,
        case when grid.com_login_cnt_per_month <-100 then 1 else grid.com_login_cnt_per_month end as com_login_cnt_per_month,
        case when grid.free_login_cnt_per_month <-100 then 1 else grid.free_login_cnt_per_month end as free_login_cnt_per_month,
        case when grid.no_ctrl_login_cnt_per_month <-100 then 1 else grid.no_ctrl_login_cnt_per_month end as no_ctrl_login_cnt_per_month,
        case when grid.strong_ctrl_login_cnt_per_month <-100 then 1 else grid.strong_ctrl_login_cnt_per_month end as strong_ctrl_login_cnt_per_month,
        case when grid.normal_ctrl_login_cnt_per_month <-100 then 1 else grid.normal_ctrl_login_cnt_per_month end as normal_ctrl_login_cnt_per_month,

        -- 上月人均引流次数（按格子统计）
        case when grid.com_base_cnt         <-100 then 0 else grid.com_base_cnt end as         com_base_cnt,
        case when grid.free_base_cnt        <-100 then 0 else grid.free_base_cnt end as        free_base_cnt,
        case when grid.no_ctrl_base_cnt     <-100 then 0 else grid.no_ctrl_base_cnt end as     no_ctrl_base_cnt,
        case when grid.strong_ctrl_base_cnt  <-100 then 0 else grid.strong_ctrl_base_cnt end as strong_ctrl_base_cnt,
        case when grid.normal_ctrl_base_cnt <-100 then 0 else grid.normal_ctrl_base_cnt end as normal_ctrl_base_cnt

    FROM
    (
    SELECT
        dt,
        user_log_acct,
        hash_sub(concat(lower(trim(user_log_acct)),'{part_dt}')) as my_hash_code,
        hash_sub(concat(lower(trim(user_log_acct)),'{part_month}')) as month_hash,
        --格子粒度
        user_life_cycle_type_1st,
        model_a_1st,
        model_b_1st,
        goal_group_1st,
        case when annual_clv_1st is null then '0' else annual_clv_1st end as annual_clv_1st,
        grid_name_1st,
        area_b_1st,

        user_life_cycle_type,
        model_a,
        model_b,
        model_c,
        model_l,
        priority_type,
        is_malice_user,
        
        -- 自然登录
        case when natural_pred_login_cnt_rest<-100 then 0
             else natural_pred_login_cnt_rest end as natural_pred_login_cnt_rest,
        case when natural_pred_login_cnt_n30d<-100 then 0
             else natural_pred_login_cnt_n30d end as natural_pred_login_cnt_n30d,

        
        --当月引流登端次数
        case when login_cnt_mtd<-100 then 0
             else login_cnt_mtd end as login_cnt_mtd,
        case when natural_login_cnt_mtd <-100 then 0 else natural_login_cnt_mtd end as natural_login_cnt_mtd,
        case when com_login_cnt_mtd <-100 then 0 else com_login_cnt_mtd end as com_login_cnt_mtd,
        case when free_login_cnt_mtd <-100 then 0 else free_login_cnt_mtd end as free_login_cnt_mtd,
        case when strong_ctrl_login_cnt_mtd <-100 then 0 else strong_ctrl_login_cnt_mtd end as strong_ctrl_login_cnt_mtd,
        case when normal_ctrl_login_cnt_mtd <-100 then 0 else normal_ctrl_login_cnt_mtd end as normal_ctrl_login_cnt_mtd,
        case when no_ctrl_login_cnt_mtd <-100 then 0 else no_ctrl_login_cnt_mtd end as no_ctrl_login_cnt_mtd,
        
        --渠道是否可达
        case when is_acce_by_com <-100 then 1 else is_acce_by_com end as is_acce_by_com,
        case when is_acce_by_free <-100 then 1 else is_acce_by_free end as is_acce_by_free,
        case when is_acce_by_strong_ctrl <-100 then 1 else is_acce_by_strong_ctrl end as is_acce_by_strong_ctrl,
        case when is_acce_by_normal_ctrl <-100 then 1 else is_acce_by_normal_ctrl end as is_acce_by_normal_ctrl,
        case when is_acce_by_no_ctrl <-100 then 1 else is_acce_by_no_ctrl end as is_acce_by_no_ctrl,
        
        --clv
        case when clv_pred_1m <-100 then 0 else clv_pred_1m end as clv_pred_1m, --0
        case when clv_mtd <-100 then 0 else clv_mtd end as clv_mtd, --0

        --上月人均干预次数
        case when login_cnt_last_month <-100 then 0 else login_cnt_last_month end                         as login_cnt_last_month,
        case when natural_login_cnt_last_month <-100 then 0 else natural_login_cnt_last_month end         as natural_login_cnt_last_month,
        case when com_login_cnt_last_month <-100 then 0 else com_login_cnt_last_month end                 as com_login_cnt_last_month,
        case when free_login_cnt_last_month <-100 then 0 else free_login_cnt_last_month end               as free_login_cnt_last_month,
        case when strong_ctrl_login_cnt_last_month<-100 then 0 else strong_ctrl_login_cnt_last_month end  as strong_ctrl_login_cnt_last_month,
        case when normal_ctrl_login_cnt_last_month<-100 then 0 else normal_ctrl_login_cnt_last_month end  as normal_ctrl_login_cnt_last_month,
        case when no_ctrl_login_cnt_last_month<-100 then 0 else no_ctrl_login_cnt_last_month end          as no_ctrl_login_cnt_last_month,
        case when clv_year<-100 then 0 else clv_year           end as gmv_year,
        case when last_year_gmv<-100 then 0 else last_year_gmv end as last_year_gmv
        
    FROM
    app.app_yhzz_umc_unit_user
    WHERE
    dt = '{part_dt}'
    and priority_type in('A1','A2' )
    limit 100
   -- and user_log_acct in (select user_log_acct from app.app_yhzz_umc_algo_pin_interim WHERE dt = '2022-05-01' and dp='low')
    )user

    JOIN

    (
    SELECT
        grid_name_1st,
        
        --阶跃下限
        max(case when lower_lift_login_cnt <-100 then 0 else lower_lift_login_cnt end) as lower_lift_login_cnt,
             

        --渠道上限
        max(case when natural_max_login_cnt <-100 then 2 else natural_max_login_cnt end )  as natural_max_login_cnt,
        max(case when com_max_login_cnt <-100 then 2 else com_max_login_cnt end )  as com_max_login_count,
        max(case when free_max_login_cnt <-100 then 2 else free_max_login_cnt end ) as free_max_login_count,
        max(case when strong_ctrl_max_login_count <-100 then 0 else strong_ctrl_max_login_count end ) as strong_ctrl_max_login_count,
        max(case when normal_ctrl_max_login_count <-100 then 0 else normal_ctrl_max_login_count end ) as normal_ctrl_max_login_count,
        max(case when no_ctrl_max_login_count <-100 then 0 else no_ctrl_max_login_count end ) as no_ctrl_max_login_count,
        
        --渠道成本
        max(case when natural_login_cost <-100 then -3 else natural_login_cost end ) as  natural_login_cost,
        max(case when com_login_cost <-100 then -3 else (-1 * com_login_cost) end ) as  com_login_cost,
        max(case when free_login_cost <-100 then 0 else free_login_cost end ) as free_login_cost,
        max(case when no_ctrl_login_cost <-100 then 0.8 else no_ctrl_login_cost end ) as no_ctrl_login_cost,
        max(case when strong_ctrl_login_cost <-100 then 0.6 else strong_ctrl_login_cost end ) as strong_ctrl_login_cost,
        max(case when normal_login_cost <-100 then 0.9 else normal_login_cost end ) as normal_login_cost,

        --渠道成本质量因子
        
        max(case when natural_quality_coeff <-100 then 1 else natural_quality_coeff end ) as natural_quality_coeff,
        max(case when com_quality_coeff <-100 then 0.28 else com_quality_coeff end ) as com_quality_coeff,
        max(case when free_quality_coeff <-100 then 0.64 else free_quality_coeff end ) as free_quality_coeff,
        max(case when strong_ctrl_quality_coeff <-100 then 2.2 else strong_ctrl_quality_coeff end ) as strong_ctrl_quality_coeff,
        max(case when normal_ctrl_quality_coeff <-100 then 0.25 else normal_ctrl_quality_coeff end ) as normal_ctrl_quality_coeff,
        max(case when no_ctrl_quality_coeff <-100 then 0.88 else no_ctrl_quality_coeff end ) as no_ctrl_quality_coeff,
        
        --渠道dau映射
        
        max(case when natural_login_cnt_per_dau <-100 then 0.478 else natural_login_cnt_per_dau end ) as natural_login_cnt_per_dau,
        max(case when com_login_cnt_per_dau <-100 then 0.460 else com_login_cnt_per_dau end ) as com_login_cnt_per_dau,
        max(case when free_login_cnt_per_dau <-100 then 0.462 else free_login_cnt_per_dau end ) as free_login_cnt_per_dau,
        max(case when no_ctrl_login_cnt_per_dau <-100 then 0.434 else no_ctrl_login_cnt_per_dau end ) as no_ctrl_login_cnt_per_dau,
        max(case when strong_ctrl_login_cnt_per_dau <-100 then 0.596 else strong_ctrl_login_cnt_per_dau end ) as strong_ctrl_login_cnt_per_dau,
        max(case when normal_login_cnt_per_dau <-100 then 0.507 else normal_login_cnt_per_dau end ) as normal_ctrl_login_cnt_per_dau,
         --dau映射
        max(case when total_login_dau_ratio <-100 then 0.8 else total_login_dau_ratio end) as total_login_dau_ratio,

        -- 格子用户数& 渠道月均引流 for 保量
        max(user_cnt) as user_cnt,
        max(natural_login_cnt_per_month) as natural_login_cnt_per_month,
        max(com_login_cnt_per_month) as com_login_cnt_per_month,
        max(free_login_cnt_per_month) as free_login_cnt_per_month,
        max(no_ctrl_login_cnt_per_month) as no_ctrl_login_cnt_per_month,
        max(strong_ctrl_login_cnt_per_month) as strong_ctrl_login_cnt_per_month,
        max(normal_ctrl_login_cnt_per_month) as normal_ctrl_login_cnt_per_month,
        max(cast(natural_login_cnt_per_month as  double)/user_cnt)         as natural_base_cnt,
        max(cast(com_login_cnt_per_month as  double)/user_cnt)         as com_base_cnt,
        max(cast(free_login_cnt_per_month as  double)/user_cnt)        as free_base_cnt,
        max(cast(no_ctrl_login_cnt_per_month as  double)/user_cnt)     as no_ctrl_base_cnt,
        max(cast(strong_ctrl_login_cnt_per_month as  double)/user_cnt) as strong_ctrl_base_cnt,
        max(cast(normal_ctrl_login_cnt_per_month as  double)/user_cnt) as normal_ctrl_base_cnt

        
    FROM
        app.app_yhzz_umc_unit_grid
    WHERE
        dt = '{part_dt}'
        AND grid_name_1st is not null
        AND is_grid_valid =1 
        group by
        grid_name_1st
    )grid on user.grid_name_1st = grid.grid_name_1st

    """.format(part_dt = part_dt,part_month = part_month)
    print("used sql", used_sql)
    data = spark.sql(used_sql)
    data.cache()
    # print("data columns", data.columns)
    print("data dtypes", data.dtypes)

    return data

In [3]:
day_ago_1 = '2022-05-20'
now_month = 5
data = get_base_data(part_dt = day_ago_1,
                     part_month= now_month)

used sql 
    SELECT
        user.dt,
        user.user_log_acct,
        user.my_hash_code,
        user.month_hash,
        --格子粒度
        user.user_life_cycle_type_1st,
        user.model_a_1st,
        user.model_b_1st,
        user.goal_group_1st,
        user.annual_clv_1st,
        user.grid_name_1st,
        user.area_b_1st,

        --格子粒度2
        user.user_life_cycle_type,
        user.model_a,
        user.model_b,
        user.model_c,
        user.model_l,
        user.priority_type,
        
        user.is_malice_user,
        
        -- 目标登录次数
        COALESCE(grid.lower_lift_login_cnt,0) as lower_lift_login_cnt,
        COALESCE(user.natural_pred_login_cnt_rest,0) as natural_pred_login_cnt_rest,
        COALESCE(user.natural_pred_login_cnt_n30d,0) as natural_pred_login_cnt_n30d,
        
        -- 自然相关
        COALESCE(user.natural_login_cnt_last_month,0) as natural_login_cnt_last_month,
        COALESCE(grid.natural_max_login_cnt,0) as natural_max_login_cnt,
      

In [4]:
test1_data = data.filter("priority_type like 'A1' or priority_type like 'A2'")
test1_data.filter( "priority_type like 'A2'").show(5)

+----------+---------------+------------+----------+------------------------+-----------+-----------+--------------+--------------+----------------------+----------+--------------------+-------+-------+-------+-------+-------------+--------------+--------------------+---------------------------+---------------------------+----------------------------+---------------------+---------------------------+---------------------+-------------------------+-------------+---------------------+-----------------+------------------+-------------------------+-------------------------+---------------------+--------------+---------------+----------------------+----------------------+------------------+-----------+----------+--------------------+------------------------+-------------------------+--------------------------------+--------------------------------+----------------------------+--------+-------------+-------------------+--------------------+---------------------------+------------------------

In [33]:
test1_data.show(5)

Py4JJavaError: An error occurred while calling o267.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2178)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2207)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2226)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:386)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectFromPlan(Dataset.scala:3397)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2558)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2558)
	at org.apache.spark.sql.Dataset$$anonfun$52.apply(Dataset.scala:3378)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withNewExecutionId$1.apply(SQLExecution.scala:81)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.

In [18]:
user_type_list = ('A1','A2','B','C')
sql = """ sel"""

In [5]:
test2_data = data.filter("priority_type like 'C'")
test2_data.show()

+---+-------------+------------+----------+------------------------+-----------+-----------+--------------+--------------+-------------+----------+--------------------+-------+-------+-------+-------+-------------+--------------+--------------------+---------------------------+---------------------------+----------------------------+---------------------+---------------------------+---------------------+-------------------------+-------------+---------------------+-----------------+------------------+-------------------------+-------------------------+---------------------+--------------+---------------+----------------------+----------------------+------------------+-----------+-------+--------------------+------------------------+-------------------------+--------------------------------+--------------------------------+----------------------------+--------+-------------+-------------------+--------------------+---------------------------+---------------------------+-----------------

In [16]:
test3_data = test1_data[['user_log_acct','my_hash_code']]
test = test1_data.unionByName(test3_data)
test.show(5)

AnalysisException: 'Cannot resolve column name "dt" among (user_log_acct, my_hash_code);'

In [14]:
# date: 2022-04-20
# updta: 2022-05-10
# author: niujianxing,yuchuchu

"""
用户管理中心-算法输出中间表
"""
import time
import datetime
from datetime import timedelta
import math
import sys

# old_time放在程序运行开始的地方
old_time = time.time()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, MapType
from pyspark.sql.functions import col, lit, array
from pyspark.sql.functions import udf

# import os
# os.environ['PYSPARK_PYTHON'] = "/usr/local/anaconda3/bin/python3.6"


spark = (SparkSession \
    .builder \
    .appName("test-dockerlinuxcontainer") \
    .enableHiveSupport() \
    .config("spark.sql.shuffle.partitions", "1000") \
    .getOrCreate())

spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")
spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "true")
spark.sql("""ADD JAR hdfs://ns1009/user/mart_jypt/mart_jypt_usr_grow/liuyang266/hiveudf-1.0-SNAPSHOT-jar-with-dependencies.jar""")
spark.sql("""CREATE TEMPORARY FUNCTION hash_sub AS 'com.jd.bdptools.ly.HashMonthly'""")

# applicationId:spark程序的唯一标识符，其格式取决于调度程序的实现
app_id = spark.sparkContext.applicationId 
print(app_id)
print(spark.version)

def get_base_data(part_dt,part_month):
    """获取sql文件

    Args:
        part_dt (_type_): 表格分区

    Returns:
        _type_: 返回执行sql
    """
    used_sql = """
    SELECT
        user.dt,
        user.user_log_acct,
        user.my_hash_code,
        user.month_hash,
        --格子粒度
        user.user_life_cycle_type_1st,
        user.model_a_1st,
        user.model_b_1st,
        user.goal_group_1st,
        user.annual_clv_1st,
        user.grid_name_1st,
        user.area_b_1st,

        --格子粒度2
        user.user_life_cycle_type,
        user.model_a,
        user.model_b,
        user.model_c,
        user.model_l,
        user.priority_type,
        
        user.is_malice_user,
        
        -- 目标登录次数
        COALESCE(grid.lower_lift_login_cnt,0) as lower_lift_login_cnt,
        COALESCE(user.natural_pred_login_cnt_rest,0) as natural_pred_login_cnt_rest,
        COALESCE(user.natural_pred_login_cnt_n30d,0) as natural_pred_login_cnt_n30d,
        
        -- 自然相关
        COALESCE(user.natural_login_cnt_last_month,0) as natural_login_cnt_last_month,
        COALESCE(grid.natural_max_login_cnt,0) as natural_max_login_cnt,
        COALESCE(grid.natural_login_cnt_per_month,0) as natural_login_cnt_per_month,
        COALESCE(grid.natural_quality_coeff,0) as natural_quality_coeff,
        COALESCE(grid.natural_login_cnt_per_dau,0) as natural_login_cnt_per_dau,

        --当月引流登端次数
        COALESCE(user.login_cnt_mtd,0) as login_cnt_mtd,
        user.natural_login_cnt_mtd,
        user.com_login_cnt_mtd,
        user.free_login_cnt_mtd,
        user.strong_ctrl_login_cnt_mtd,
        user.normal_ctrl_login_cnt_mtd,
        user.no_ctrl_login_cnt_mtd,

        --渠道是否可达
        user.is_acce_by_com,
        user.is_acce_by_free,
        user.is_acce_by_strong_ctrl,
        user.is_acce_by_normal_ctrl,
        user.is_acce_by_no_ctrl,

        --clv
        user.clv_pred_1m,
        user.clv_mtd,

        --上月人均干预次数
        COALESCE(login_cnt_last_month,0)         as login_cnt_last_month,
        COALESCE(com_login_cnt_last_month,0) as com_login_cnt_last_month,
        COALESCE(free_login_cnt_last_month,0) as free_login_cnt_last_month,
        COALESCE(strong_ctrl_login_cnt_last_month,0) as strong_ctrl_login_cnt_last_month,
        COALESCE(normal_ctrl_login_cnt_last_month,0) as normal_ctrl_login_cnt_last_month,
        COALESCE(no_ctrl_login_cnt_last_month,0) as no_ctrl_login_cnt_last_month,
        
        --gmv
        cast(COALESCE(gmv_year,0.0) as double)  as gmv_year,
        cast(COALESCE(last_year_gmv,0.0) as double) as last_year_gmv,
                
        --渠道上限
        COALESCE(grid.com_max_login_count,2) as com_max_login_count,
        COALESCE(grid.free_max_login_count,2) as free_max_login_count,
        COALESCE(grid.strong_ctrl_max_login_count,0) as strong_ctrl_max_login_count,
        COALESCE(grid.normal_ctrl_max_login_count,0) as normal_ctrl_max_login_count,
        COALESCE(grid.no_ctrl_max_login_count,0) as no_ctrl_max_login_count,

        --渠道成本
        COALESCE(grid.com_login_cost,-3) as com_login_cost,
        COALESCE(grid.free_login_cost,0) as free_login_cost,
        COALESCE(grid.no_ctrl_login_cost,0.9) as no_ctrl_login_cost,
        COALESCE(grid.strong_ctrl_login_cost,0.8) as strong_ctrl_login_cost,
        COALESCE(grid.normal_login_cost,0.6) as normal_login_cost,

        --渠道成本质量因子
        com_quality_coeff,
        free_quality_coeff,
        strong_ctrl_quality_coeff,
        normal_ctrl_quality_coeff,
        no_ctrl_quality_coeff,


        --渠道dau映射
        COALESCE(grid.com_login_cnt_per_dau,0.5)            as com_login_cnt_per_dau,
        COALESCE(grid.free_login_cnt_per_dau,0.5)           as free_login_cnt_per_dau,
        COALESCE(grid.no_ctrl_login_cnt_per_dau,0.5)        as no_ctrl_login_cnt_per_dau,
        COALESCE(grid.strong_ctrl_login_cnt_per_dau,0.5)    as strong_ctrl_login_cnt_per_dau,
        COALESCE(grid.normal_ctrl_login_cnt_per_dau,0.5)         as normal_ctrl_login_cnt_per_dau,
        grid.total_login_dau_ratio,

        -- 格子用户数& 渠道月均引流 for 保量
        case when grid.user_cnt <-100 then 1 else grid.user_cnt end as user_cnt,
        case when grid.com_login_cnt_per_month <-100 then 1 else grid.com_login_cnt_per_month end as com_login_cnt_per_month,
        case when grid.free_login_cnt_per_month <-100 then 1 else grid.free_login_cnt_per_month end as free_login_cnt_per_month,
        case when grid.no_ctrl_login_cnt_per_month <-100 then 1 else grid.no_ctrl_login_cnt_per_month end as no_ctrl_login_cnt_per_month,
        case when grid.strong_ctrl_login_cnt_per_month <-100 then 1 else grid.strong_ctrl_login_cnt_per_month end as strong_ctrl_login_cnt_per_month,
        case when grid.normal_ctrl_login_cnt_per_month <-100 then 1 else grid.normal_ctrl_login_cnt_per_month end as normal_ctrl_login_cnt_per_month,

        -- 上月人均引流次数（按格子统计）
        case when grid.com_base_cnt         <-100 then 0 else grid.com_base_cnt end as         com_base_cnt,
        case when grid.free_base_cnt        <-100 then 0 else grid.free_base_cnt end as        free_base_cnt,
        case when grid.no_ctrl_base_cnt     <-100 then 0 else grid.no_ctrl_base_cnt end as     no_ctrl_base_cnt,
        case when grid.strong_ctrl_base_cnt  <-100 then 0 else grid.strong_ctrl_base_cnt end as strong_ctrl_base_cnt,
        case when grid.normal_ctrl_base_cnt <-100 then 0 else grid.normal_ctrl_base_cnt end as normal_ctrl_base_cnt

    FROM
    (
    SELECT
        dt,
        user_log_acct,
        hash_sub(concat(lower(trim(user_log_acct)),'{part_dt}')) as my_hash_code,
        hash_sub(concat(lower(trim(user_log_acct)),'{part_month}')) as month_hash,
        --格子粒度
        user_life_cycle_type_1st,
        model_a_1st,
        model_b_1st,
        goal_group_1st,
        case when annual_clv_1st is null then '0' else annual_clv_1st end as annual_clv_1st,
        grid_name_1st,
        area_b_1st,

        user_life_cycle_type,
        model_a,
        model_b,
        model_c,
        model_l,
        priority_type,
        is_malice_user,
        
        -- 自然登录
        case when natural_pred_login_cnt_rest<-100 then 0
             else natural_pred_login_cnt_rest end as natural_pred_login_cnt_rest,
        case when natural_pred_login_cnt_n30d<-100 then 0
             else natural_pred_login_cnt_n30d end as natural_pred_login_cnt_n30d,

        
        --当月引流登端次数
        case when login_cnt_mtd<-100 then 0
             else login_cnt_mtd end as login_cnt_mtd,
        case when natural_login_cnt_mtd <-100 then 0 else natural_login_cnt_mtd end as natural_login_cnt_mtd,
        case when com_login_cnt_mtd <-100 then 0 else com_login_cnt_mtd end as com_login_cnt_mtd,
        case when free_login_cnt_mtd <-100 then 0 else free_login_cnt_mtd end as free_login_cnt_mtd,
        case when strong_ctrl_login_cnt_mtd <-100 then 0 else strong_ctrl_login_cnt_mtd end as strong_ctrl_login_cnt_mtd,
        case when normal_ctrl_login_cnt_mtd <-100 then 0 else normal_ctrl_login_cnt_mtd end as normal_ctrl_login_cnt_mtd,
        case when no_ctrl_login_cnt_mtd <-100 then 0 else no_ctrl_login_cnt_mtd end as no_ctrl_login_cnt_mtd,
        
        --渠道是否可达
        case when is_acce_by_com <-100 then 1 else is_acce_by_com end as is_acce_by_com,
        case when is_acce_by_free <-100 then 1 else is_acce_by_free end as is_acce_by_free,
        case when is_acce_by_strong_ctrl <-100 then 1 else is_acce_by_strong_ctrl end as is_acce_by_strong_ctrl,
        case when is_acce_by_normal_ctrl <-100 then 1 else is_acce_by_normal_ctrl end as is_acce_by_normal_ctrl,
        case when is_acce_by_no_ctrl <-100 then 1 else is_acce_by_no_ctrl end as is_acce_by_no_ctrl,
        
        --clv
        case when clv_pred_1m <-100 then 0 else clv_pred_1m end as clv_pred_1m, --0
        case when clv_mtd <-100 then 0 else clv_mtd end as clv_mtd, --0

        --上月人均干预次数
        case when login_cnt_last_month <-100 then 0 else login_cnt_last_month end                         as login_cnt_last_month,
        case when natural_login_cnt_last_month <-100 then 0 else natural_login_cnt_last_month end         as natural_login_cnt_last_month,
        case when com_login_cnt_last_month <-100 then 0 else com_login_cnt_last_month end                 as com_login_cnt_last_month,
        case when free_login_cnt_last_month <-100 then 0 else free_login_cnt_last_month end               as free_login_cnt_last_month,
        case when strong_ctrl_login_cnt_last_month<-100 then 0 else strong_ctrl_login_cnt_last_month end  as strong_ctrl_login_cnt_last_month,
        case when normal_ctrl_login_cnt_last_month<-100 then 0 else normal_ctrl_login_cnt_last_month end  as normal_ctrl_login_cnt_last_month,
        case when no_ctrl_login_cnt_last_month<-100 then 0 else no_ctrl_login_cnt_last_month end          as no_ctrl_login_cnt_last_month,
        case when clv_year<-100 then 0 else clv_year           end as gmv_year,
        case when last_year_gmv<-100 then 0 else last_year_gmv end as last_year_gmv
        
    FROM
    app.app_yhzz_umc_unit_user
    WHERE
    dt = '{part_dt}'
    and priority_type in('A1','A2' )
   -- and user_log_acct in (select user_log_acct from app.app_yhzz_umc_algo_pin_interim WHERE dt = '2022-05-01' and dp='low')
    )user

    JOIN

    (
    SELECT
        grid_name_1st,
        
        --阶跃下限
        max(case when lower_lift_login_cnt <-100 then 0 else lower_lift_login_cnt end) as lower_lift_login_cnt,
             

        --渠道上限
        max(case when natural_max_login_cnt <-100 then 2 else natural_max_login_cnt end )  as natural_max_login_cnt,
        max(case when com_max_login_cnt <-100 then 2 else com_max_login_cnt end )  as com_max_login_count,
        max(case when free_max_login_cnt <-100 then 2 else free_max_login_cnt end ) as free_max_login_count,
        max(case when strong_ctrl_max_login_count <-100 then 0 else strong_ctrl_max_login_count end ) as strong_ctrl_max_login_count,
        max(case when normal_ctrl_max_login_count <-100 then 0 else normal_ctrl_max_login_count end ) as normal_ctrl_max_login_count,
        max(case when no_ctrl_max_login_count <-100 then 0 else no_ctrl_max_login_count end ) as no_ctrl_max_login_count,
        
        --渠道成本
        max(case when natural_login_cost <-100 then -3 else natural_login_cost end ) as  natural_login_cost,
        max(case when com_login_cost <-100 then -3 else (-1 * com_login_cost) end ) as  com_login_cost,
        max(case when free_login_cost <-100 then 0 else free_login_cost end ) as free_login_cost,
        max(case when no_ctrl_login_cost <-100 then 0.8 else no_ctrl_login_cost end ) as no_ctrl_login_cost,
        max(case when strong_ctrl_login_cost <-100 then 0.6 else strong_ctrl_login_cost end ) as strong_ctrl_login_cost,
        max(case when normal_login_cost <-100 then 0.9 else normal_login_cost end ) as normal_login_cost,

        --渠道成本质量因子
        
        max(case when natural_quality_coeff <-100 then 1 else natural_quality_coeff end ) as natural_quality_coeff,
        max(case when com_quality_coeff <-100 then 0.28 else com_quality_coeff end ) as com_quality_coeff,
        max(case when free_quality_coeff <-100 then 0.64 else free_quality_coeff end ) as free_quality_coeff,
        max(case when strong_ctrl_quality_coeff <-100 then 2.2 else strong_ctrl_quality_coeff end ) as strong_ctrl_quality_coeff,
        max(case when normal_ctrl_quality_coeff <-100 then 0.25 else normal_ctrl_quality_coeff end ) as normal_ctrl_quality_coeff,
        max(case when no_ctrl_quality_coeff <-100 then 0.88 else no_ctrl_quality_coeff end ) as no_ctrl_quality_coeff,
        
        --渠道dau映射
        
        max(case when natural_login_cnt_per_dau <-100 then 0.478 else natural_login_cnt_per_dau end ) as natural_login_cnt_per_dau,
        max(case when com_login_cnt_per_dau <-100 then 0.460 else com_login_cnt_per_dau end ) as com_login_cnt_per_dau,
        max(case when free_login_cnt_per_dau <-100 then 0.462 else free_login_cnt_per_dau end ) as free_login_cnt_per_dau,
        max(case when no_ctrl_login_cnt_per_dau <-100 then 0.434 else no_ctrl_login_cnt_per_dau end ) as no_ctrl_login_cnt_per_dau,
        max(case when strong_ctrl_login_cnt_per_dau <-100 then 0.596 else strong_ctrl_login_cnt_per_dau end ) as strong_ctrl_login_cnt_per_dau,
        max(case when normal_login_cnt_per_dau <-100 then 0.507 else normal_login_cnt_per_dau end ) as normal_ctrl_login_cnt_per_dau,
         --dau映射
        max(case when total_login_dau_ratio <-100 then 0.8 else total_login_dau_ratio end) as total_login_dau_ratio,

        -- 格子用户数& 渠道月均引流 for 保量
        max(user_cnt) as user_cnt,
        max(natural_login_cnt_per_month) as natural_login_cnt_per_month,
        max(com_login_cnt_per_month) as com_login_cnt_per_month,
        max(free_login_cnt_per_month) as free_login_cnt_per_month,
        max(no_ctrl_login_cnt_per_month) as no_ctrl_login_cnt_per_month,
        max(strong_ctrl_login_cnt_per_month) as strong_ctrl_login_cnt_per_month,
        max(normal_ctrl_login_cnt_per_month) as normal_ctrl_login_cnt_per_month,
        max(cast(natural_login_cnt_per_month as  double)/user_cnt)         as natural_base_cnt,
        max(cast(com_login_cnt_per_month as  double)/user_cnt)         as com_base_cnt,
        max(cast(free_login_cnt_per_month as  double)/user_cnt)        as free_base_cnt,
        max(cast(no_ctrl_login_cnt_per_month as  double)/user_cnt)     as no_ctrl_base_cnt,
        max(cast(strong_ctrl_login_cnt_per_month as  double)/user_cnt) as strong_ctrl_base_cnt,
        max(cast(normal_ctrl_login_cnt_per_month as  double)/user_cnt) as normal_ctrl_base_cnt

        
    FROM
        app.app_yhzz_umc_unit_grid
    WHERE
        dt = '{part_dt}'
        AND grid_name_1st is not null
        AND is_grid_valid =1 
        group by
        grid_name_1st
    )grid on user.grid_name_1st = grid.grid_name_1st

    """.format(part_dt = part_dt,part_month = part_month)
    print("used sql", used_sql)
    data = spark.sql(used_sql)
    data.cache()
    print("data columns", data.columns)
    print("data dtypes", data.dtypes)

    return data

def get_plan_pra(plan_level):
    """返回不同档位的参数值

    Args:
        plan_level: 方案档位, low, medium, high, max

    Returns:
        _type_: 返回月度clv阈值(month_clv_threshold), 
                返回年度clv阈值year_clv_threshold, 
                返回渠道放大系数channel_max_adjust, 
                返回渠道偏好系数channel_preference
    """
    # 月度clv阈值
    # 年度clv等级
    # 渠道上限调节, 999表示取上限，1.5表示均值的1.5倍，1.2表示均值的1.2倍，等等
    # 渠道倾向 channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
    
    # plan_pra_dict = {'max' : [50, '1', 999, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'high' : [50, '1', 1.5, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'medium' : [50, '1', 1.2, [1.0, 1.0, 1.0, 1.0,1.0]],
    #                  'low' : [50, '1', 1.0, [1.0, 1.0, 1.0, 1.0,1.0]]}
    
    if plan_level == 'max':
        month_clv_threshold = 50  
        year_clv_threshold = '1'  
        channel_max_adjust = 999  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    elif plan_level == 'high':
        month_clv_threshold = 50
        year_clv_threshold = '1'  
        channel_max_adjust = 1.5 
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    elif plan_level == 'medium':
        month_clv_threshold = 50  
        year_clv_threshold = '1'  
        channel_max_adjust = 1.2  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  
    else:
        plan_level = 'lowv3'
        month_clv_threshold = 50  
        year_clv_threshold = '1' 
        channel_max_adjust = 1.0  
        channel_preference = [1.0, 1.0, 1.0, 1.0,
                              1.0]  

    return month_clv_threshold, year_clv_threshold, channel_max_adjust, channel_preference

def add_channel_result_col(data, this_month_end):
    """数据新增列

    Args:
        data(dataframe): 基础数据

    Returns:
        dataframe: 新增分配结果列 的数据
    """
    channel_type = ["com", "free", "strong_ctrl", "normal_ctrl", "no_ctrl"]
    data = data.withColumn("pred_total_dau",lit(0.0))
    for channel_name in channel_type:
        # 干预次数
        data = data.withColumn('%s_login_cnt'%channel_name, 
                                         data["result"].getItem(channel_name))
        # DAU
        data = data.withColumn("pred_%s_dau"%channel_name, 
                                         col("%s_login_cnt"%channel_name) * col("%s_login_cnt_per_dau"%channel_name))
        # total_DAU
        data = data.withColumn("pred_total_dau",
                                         col("pred_total_dau") + col("%s_login_cnt_per_dau"%channel_name))
    # 增加自然端的数据
    data = data.withColumn("pred_natural_dau",col("natural_pred_login_cnt_rest") * col("natural_login_cnt_per_dau"))
    data = data.withColumn("pred_total_dau",col("pred_total_dau") + col("pred_natural_dau"))
    

    data = data.withColumn("pred_total_dau", col("pred_total_dau") * col("total_login_dau_ratio"))
    data = data.withColumn('is_solvable', data["result"].getItem("flag"))

    # TODO 优先级 P1
    data = data.withColumn('priority_level', lit(50))
    data = data.withColumn("end_time", lit(this_month_end))  # 本月最后1s
    
    # TODO 保量相关信息传出
    data = data.withColumn('is_guaranteed_by_com', lit(1))
    data = data.withColumn('is_guaranteed_by_free', lit(1))
    data = data.withColumn('is_guaranteed_by_strong_ctrl', lit(1))
    data = data.withColumn('is_guaranteed_by_normal_ctrl', lit(1))
    data = data.withColumn('is_guaranteed_by_no_ctrl', lit(1))
    
   
    # 算法中间值
    algo_total_info = """{"guaranteed":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0},
                            "channel_max_login_days":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0},
                            "channel_min_login_days":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0},
                            "channel_login_costs":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0},
                            "greedy_result":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0},
                            "result":{"com": 0, "free": 0, "strong_ctrl": 0, "normal_ctrl": 0, "no_ctrl": 0}}"""
    
    data = data.withColumn("algo_total_info", lit(algo_total_info)) 
    
    # 字段名称修正
    rename_cols_dict = {'natural_login_cnt_last_month':'natural_user_last_month_login_cnt',\
                        'com_login_cnt_last_month':'com_user_last_month_login_cnt',\
                        'free_login_cnt_last_month':'free_user_last_month_login_cnt',\
                        'strong_ctrl_login_cnt_last_month':'strong_ctrl_user_last_month_login_cnt',\
                        'normal_ctrl_login_cnt_last_month':'normal_ctrl_user_last_month_login_cnt',\
                        'no_ctrl_login_cnt_last_month':'no_ctrl_user_last_month_login_cnt',\
                        
                        'is_guaranteed_by_com':'com_user_base_guarteed_cnt',\
                        'is_guaranteed_by_free':'free_user_base_guarteed_cnt',\
                        'is_guaranteed_by_strong_ctrl':'strong_ctrl_user_base_guarteed_cnt',\
                        'is_guaranteed_by_normal_ctrl':'normal_ctrl_user_base_guarteed_cnt',\
                        'is_guaranteed_by_no_ctrl':'no_ctrl_user_base_guarteed_cnt',\
                        'is_acce_by_com':'com_user_acce_flag',\
                        'is_acce_by_free':'free_user_acce_flag',\
                        'is_acce_by_strong_ctrl':'strong_ctrl_user_acce_flag',\
                        'is_acce_by_normal_ctrl':'normal_ctrl_user_acce_flag',\
                        'is_acce_by_no_ctrl':'no_ctrl_user_acce_flag',\
                            
                        'natural_max_login_cnt':'natural_grid_max_login_cnt',\
                        'com_max_login_count':'com_grid_max_login_cnt',\
                        'free_max_login_count':'free_grid_max_login_cnt',\
                        'strong_ctrl_max_login_count':'strong_ctrl_grid_max_login_cnt',\
                        'normal_ctrl_max_login_count':'normal_ctrl_grid_max_login_cnt',\
                        'no_ctrl_max_login_count':'no_ctrl_grid_max_login_cnt',\
                            
                        'natural_login_cnt_per_month':'natural_grid_last_month_total_cnt',\
                        'com_login_cnt_per_month':'com_grid_last_month_total_cnt',\
                        'free_login_cnt_per_month':'free_grid_last_month_total_cnt',\
                        'strong_ctrl_login_cnt_per_month':'strong_ctrl_grid_last_month_total_cnt',\
                        'normal_ctrl_login_cnt_per_month':'normal_ctrl_grid_last_month_total_cnt',\
                        'no_ctrl_login_cnt_per_month':'no_ctrl_grid_last_month_total_cnt',\
                            
                        'natural_login_cost':'natural_last_login_cost',\
                        'com_login_cost':'com_grid_last_login_cost',\
                        'free_login_cost':'free_grid_last_login_cost',\
                        'strong_ctrl_login_cost':'strong_ctrl_grid_last_login_cost',\
                        'normal_login_cost':'normal_ctrl_grid_last_login_cost',\
                        'no_ctrl_login_cost':'no_ctrl_grid_last_login_cost',\
                            
                        'natural_quality_coeff':'natural_grid_quality_coeff',\
                        'com_quality_coeff':'com_grid_quality_coeff',\
                        'free_quality_coeff':'free_grid_quality_coeff',\
                        'strong_ctrl_quality_coeff':'strong_ctrl_grid_quality_coeff',\
                        'normal_ctrl_quality_coeff':'normal_ctrl_grid_quality_coeff',\
                        'no_ctrl_quality_coeff':'no_ctrl_grid_quality_coeff',\
                        
                        'natural_login_cnt_per_dau':'natural_grid_login_cnt_per_dau',\
                        'com_login_cnt_per_dau':'com_grid_login_cnt_per_dau',\
                        'free_login_cnt_per_dau':'free_grid_login_cnt_per_dau',\
                        'strong_ctrl_login_cnt_per_dau':'strong_ctrl_grid_login_cnt_per_dau',\
                        'normal_ctrl_login_cnt_per_dau':'normal_grid_login_cnt_per_dau',\
                        'no_ctrl_login_cnt_per_dau':'no_ctrl_grid_login_cnt_per_dau'}
    
    print('rename_cols_dict',rename_cols_dict)
    print(data.columns)

    if isinstance(rename_cols_dict, dict):
        for old_name, new_name in rename_cols_dict.items():
            if old_name in list(data.columns):
                print(old_name, new_name)
                data = data.withColumnRenamed(old_name, new_name)
            else:
                print("%s not in data"%old_name)
    else:
        raise ValueError("'columns' should be a dict, like {'old_name_1':'new_name_1', 'old_name_2':'new_name_2'}")
    
    
    return data

def data_insert_table(data ,table_name ,table_dt ,tabel_dp):
    table_cols = ['user_log_acct','user_life_cycle_type_1st','model_a_1st','model_b_1st','goal_group_1st','annual_clv_1st','grid_name_1st','clv_pred_1m',
                  'model_a','model_b','model_c','model_l','user_life_cycle_type',
                  'login_cnt_mtd','natural_login_cnt_mtd','com_login_cnt_mtd','free_login_cnt_mtd',
                  'strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                  'com_user_last_month_login_cnt','free_user_last_month_login_cnt',
                  'strong_ctrl_user_last_month_login_cnt','normal_ctrl_user_last_month_login_cnt',
                  'no_ctrl_user_last_month_login_cnt','com_user_base_guarteed_cnt','free_user_base_guarteed_cnt',
                  'strong_ctrl_user_base_guarteed_cnt','normal_ctrl_user_base_guarteed_cnt','no_ctrl_user_base_guarteed_cnt',
                  'com_user_acce_flag','free_user_acce_flag',
                  'strong_ctrl_user_acce_flag','normal_ctrl_user_acce_flag','no_ctrl_user_acce_flag',
                  'com_grid_max_login_cnt','free_grid_max_login_cnt',
                  'strong_ctrl_grid_max_login_cnt','normal_ctrl_grid_max_login_cnt','no_ctrl_grid_max_login_cnt',
                  'com_grid_last_month_total_cnt','free_grid_last_month_total_cnt','strong_ctrl_grid_last_month_total_cnt',
                  'normal_ctrl_grid_last_month_total_cnt','no_ctrl_grid_last_month_total_cnt',
                  'com_grid_quality_coeff','free_grid_quality_coeff','strong_ctrl_grid_quality_coeff',
                  'normal_ctrl_grid_quality_coeff','no_ctrl_grid_quality_coeff',  
                  'com_grid_login_cnt_per_dau','free_grid_login_cnt_per_dau','strong_ctrl_grid_login_cnt_per_dau',
                  'normal_grid_login_cnt_per_dau','no_ctrl_grid_login_cnt_per_dau',
                  'com_grid_last_login_cost','free_grid_last_login_cost','strong_ctrl_grid_last_login_cost',
                  'normal_ctrl_grid_last_login_cost','no_ctrl_grid_last_login_cost',
                  'algo_total_info','natural_pred_login_cnt_rest',
                  'com_login_cnt','free_login_cnt','strong_ctrl_login_cnt','normal_ctrl_login_cnt','no_ctrl_login_cnt',
                  'priority_level','pred_com_dau','pred_free_dau','pred_strong_ctrl_dau','pred_normal_ctrl_dau','pred_no_ctrl_dau',
                  'total_login_dau_ratio','pred_total_dau','is_solvable','end_time','is_malice_user','pred_natural_dau','priority_type'] # 修正1 增加恶意用户 增加pred_natural_dau 
    print('table_cols', table_cols)
    result_data = data[table_cols]
    result_data.createOrReplaceTempView('result_data_tmp')

    
    sql = """
       INSERT OVERWRITE TABLE {table_name} PARTITION (dt='{part_dt}', priority_type, dp)

        select
         *,
         '{part_dp}' as dp
        from
        result_data_tmp
    """.format(table_name=table_name, part_dt=table_dt, part_dp=tabel_dp)
    print(sql)
    spark.sql(sql)
    print("successful data insert tabel")
    return 

def juge_how_allocation(run_day, base_data):
    
    # TODO 5月临时方案 修改为整体方案
    first_allocation_day = datetime.datetime.strptime('2022-05-15','%Y-%M-%d')
    run_day = datetime.datetime.strptime(run_day,'%Y-%M-%d')
    day1_ago_runday = datetime.datetime.strftime(run_day- timedelta(days=1),'%Y-%M-%d')
    
    delta_day = (run_day-first_allocation_day).days
    
    if delta_day == 0:
        #  首日分配
        print(">"*10,"进行首日分配") 
        final_data = base_data.withColumn('result',get_allocation_result('dt','month_clv_threshold','year_clv_threshold','channel_max_adjust',
                                                         'channel_preference','a1_threshold','priority_type','my_hash_code',
                                                         'lower_lift_login_cnt','natural_pred_login_cnt_rest','login_cnt_mtd','natural_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                                                         'is_acce_by_com','is_acce_by_free','is_acce_by_strong_ctrl',
                                                         'is_acce_by_normal_ctrl','is_acce_by_no_ctrl',
                                                         'com_max_login_count','free_max_login_count','strong_ctrl_max_login_count',
                                                         'normal_ctrl_max_login_count','no_ctrl_max_login_count',
                                                         'com_login_cost','free_login_cost','strong_ctrl_login_cost','normal_login_cost',
                                                         'no_ctrl_login_cost',
                                                         'com_quality_coeff','free_quality_coeff','strong_ctrl_quality_coeff',
                                                         'normal_ctrl_quality_coeff','no_ctrl_quality_coeff',
                                                         'com_login_cnt_per_dau','free_login_cnt_per_dau',
                                                         'strong_ctrl_login_cnt_per_dau','normal_ctrl_login_cnt_per_dau',
                                                         'no_ctrl_login_cnt_per_dau',
                                                         'clv_pred_1m','gmv_year',
                                                         'login_cnt_last_month','natural_login_cnt_last_month','com_login_cnt_last_month','free_login_cnt_last_month','strong_ctrl_login_cnt_last_month','normal_ctrl_login_cnt_last_month','no_ctrl_login_cnt_last_month',
                                                         'com_login_cnt_mtd','free_login_cnt_mtd','strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','month_hash'))

    elif delta_day%3 == 0:
        # TODO 高活和低活的调整周期不一样 现在只写了高活
        # 3日-高活 渠道内调整 执行首日分配算法
        print(">"*10,"进行高活用户 渠道内调整") 
        final_data = base_data.withColumn('result',get_allocation_result('dt','month_clv_threshold','year_clv_threshold','channel_max_adjust',
                                                         'channel_preference','a1_threshold','priority_type','my_hash_code',
                                                         'lower_lift_login_cnt','natural_pred_login_cnt_rest','login_cnt_mtd','natural_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                                                         'is_acce_by_com','is_acce_by_free','is_acce_by_strong_ctrl',
                                                         'is_acce_by_normal_ctrl','is_acce_by_no_ctrl',
                                                         'com_max_login_count','free_max_login_count','strong_ctrl_max_login_count',
                                                         'normal_ctrl_max_login_count','no_ctrl_max_login_count',
                                                         'com_login_cost','free_login_cost','strong_ctrl_login_cost','normal_login_cost',
                                                         'no_ctrl_login_cost',
                                                         'com_quality_coeff','free_quality_coeff','strong_ctrl_quality_coeff',
                                                         'normal_ctrl_quality_coeff','no_ctrl_quality_coeff',
                                                         'com_login_cnt_per_dau','free_login_cnt_per_dau',
                                                         'strong_ctrl_login_cnt_per_dau','normal_ctrl_login_cnt_per_dau',
                                                         'no_ctrl_login_cnt_per_dau',
                                                         'clv_pred_1m','gmv_year',
                                                         'login_cnt_last_month','natural_login_cnt_last_month','com_login_cnt_last_month','free_login_cnt_last_month','strong_ctrl_login_cnt_last_month','normal_ctrl_login_cnt_last_month','no_ctrl_login_cnt_last_month',
                                                         'com_login_cnt_mtd','free_login_cnt_mtd','strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','month_hash'))
   
    elif (delta_day==7) or (delta_day-7)%6==0:
        # TODO 无渠道间调整 重复取数 是否可优化
        # 7日 监控期，然后高活6d周期 渠道间调整
        print(">"*10,"进行高活用户 渠道间调整") 
        lastday_data = get_lastday_data_no_allocation(dp, day1_ago_runday)
        final_data = lastday_data
    
    elif delta_day%3 > 0:
        # 非首日 非调控期 取昨日分区减去昨日已完成
        print(">"*10,"非首日，非调控期，取昨日分区减去昨日已完成") 
        lastday_data = get_lastday_data_no_allocation(dp, day1_ago_runday)
        final_data = lastday_data
    
    return final_data

@udf(returnType=MapType(StringType(), IntegerType()))
def get_allocation_result(part_dt,month_clv_threshold,year_clv_threshold,channel_max_adjust,channel_preference,a1_threshold,priority_type,my_hash_code,
                lower_lift_login_cnt, natural_pred_login_cnt_rest,login_cnt_mtd,natural_login_cnt_mtd,no_ctrl_login_cnt_mtd,
                is_acce_by_com, is_acce_by_free, is_acce_by_strong_ctrl, is_acce_by_normal_ctrl,is_acce_by_no_ctrl,
                com_max_login_count, free_max_login_count, strong_ctrl_max_login_count,normal_ctrl_max_login_count,no_ctrl_max_login_count,
                com_login_cost, free_login_cost, strong_ctrl_login_cost, normal_login_cost,no_ctrl_login_cost, 
                com_quality_coeff,free_quality_coeff,strong_ctrl_quality_coeff,normal_ctrl_quality_coeff,no_ctrl_quality_coeff,
                com_login_cnt_per_dau, free_login_cnt_per_dau,  strong_ctrl_login_cnt_per_dau, normal_ctrl_login_cnt_per_dau, no_ctrl_login_cnt_per_dau,
                clv_pred_1m,gmv_year,
                login_cnt_last_month, natural_login_cnt_last_month,com_login_cnt_last_month,free_login_cnt_last_month,strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month,
                com_login_cnt_mtd,free_login_cnt_mtd,strong_ctrl_login_cnt_mtd,normal_ctrl_login_cnt_mtd,month_hash):
    

    # 用户价值
    clv = get_user_clv(clv_pred_1m)
    user_label = priority_type
    
    # 第一层 基础保量
    basic_result = BasicGuarantee(com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                                  normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month)
    
    # 第二层 业务规则  输出业务结果以及对阶跃渠道上限处理
    businessrule_result, businessrule_no_allocation_channel, channel_businessrule_max, channel_businessrule_max_add = BusinessRuleSupplement(basic_result, user_label,  year_clv_threshold, gmv_year, clv, month_clv_threshold, 
                                                com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                                                normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month, 
                                                my_hash_code, month_hash)
    
    # 第三层 贪心求解阶跃
    # 目标 TODO 获取目标 渠道上限 以及成本写为一个类
    # part_dt,lift_login, natura_login, current_login,no_ctrl_login
    channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
    target_login_days = get_target_allocation_login(part_dt=part_dt,
                                                    lift_login=lower_lift_login_cnt,
                                                    natura_login=natural_pred_login_cnt_rest , #用上月自然登录次数替代本月预测v4 (natural_login_cnt_last_month - natural_login_cnt_mtd)
                                                    current_login=login_cnt_mtd,
                                                    no_ctrl_login=(no_ctrl_login_cnt_last_month - no_ctrl_login_cnt_mtd))

    # 获取阶跃渠道上限 & 参数调整（业务加量以及渠道放大）
    channel_max_login_days   = get_user_channel_max_func(priority_type, my_hash_code, channel_businessrule_max_add, channel_businessrule_max,  
                                                            com_max_login_count, free_max_login_count, strong_ctrl_max_login_count, normal_ctrl_max_login_count,no_ctrl_max_login_count,
                                                            com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month, normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month,
                                                            is_acce_by_com, is_acce_by_free, is_acce_by_strong_ctrl, is_acce_by_normal_ctrl, is_acce_by_no_ctrl,
                                                            month_clv_threshold,channel_max_adjust,channel_preference)
    # 渠道成本
    channel_login_costs = get_channel_login_costs(channel_type, com_login_cost, free_login_cost, strong_ctrl_login_cost, normal_login_cost, no_ctrl_login_cost,
                                                  com_login_cnt_per_dau, free_login_cnt_per_dau, strong_ctrl_login_cnt_per_dau, normal_ctrl_login_cnt_per_dau, no_ctrl_login_cnt_per_dau,
                                                  com_quality_coeff, free_quality_coeff, strong_ctrl_quality_coeff, normal_ctrl_quality_coeff, no_ctrl_quality_coeff)

    # 求解
    greedy_result = GreedySolver(target_login_days, businessrule_no_allocation_channel, businessrule_result, 
                                 channel_max_login_days,  channel_login_costs)

    # 根据用户类型修正渠道分配方案    
    # 结果最终修正1 不小于上月用户次数  
    final_result = greedy_result
    final_result['com'] = math.ceil(max(final_result['com'],com_login_cnt_last_month))
    final_result['free'] = math.ceil(max(final_result['free'],free_login_cnt_last_month))
    final_result['strong_ctrl'] = math.ceil(max(final_result['strong_ctrl'],strong_ctrl_login_cnt_last_month))
    final_result['no_ctrl'] = math.ceil(max(final_result['no_ctrl'],no_ctrl_login_cnt_last_month))
    
    ## 修正2 可控置零
    if user_label == 'A1':
        final_result['normal_ctrl'] = 0
        # # 短信渠道补量
        # if (my_hash_code <= 5):
        #     final_result['strong_ctrl'] += 1
    else:
        final_result['normal_ctrl'] = math.ceil(max(final_result['normal_ctrl'], normal_ctrl_login_cnt_last_month))
        final_result['normal_ctrl'] = math.ceil(max(final_result['normal_ctrl'] - normal_ctrl_login_cnt_mtd, 0))
    
    # 结果最终修正3 减去渠道已完成
    final_result['com'] = math.ceil(max(final_result['com'] - com_login_cnt_mtd, 1))
    final_result['free'] = math.ceil(max(final_result['free'] - free_login_cnt_mtd, 0))
    final_result['strong_ctrl'] = math.ceil(max(final_result['strong_ctrl'] - strong_ctrl_login_cnt_mtd, 0))
    final_result['no_ctrl'] = math.ceil(max(final_result['no_ctrl'] - no_ctrl_login_cnt_mtd, 0))    

    re = sum(list(final_result.values()))
    target_flag = 1 if re >= target_login_days else 0
    final_result['flag'] = target_flag       
    return final_result

def get_user_clv(clv_pred_1m):
    """_summary_

    Args:
        clv_pred_1m (_type_): _description_

    Returns:
        _type_: _description_
    """
    return max(clv_pred_1m,0)

# 返回值必须全为Int, 否则类型不对的变量会被设为null
# @udf(returnType=MapType(StringType(), IntegerType()))
def BasicGuarantee(com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month, normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month):
    """获取渠道保量(用户上月该渠道引流成功次数,向上取整),渠道保量作为用户渠引流成功下限

    Args:
        com_login_cnt_last_month (_type_): _description_
        free_login_cnt_last_month (_type_): _description_
        strong_ctrl_login_cnt_last_month (_type_): _description_
        normal_ctrl_login_cnt_last_month (_type_): _description_
        no_ctrl_login_cnt_last_month (_type_): _description_

    Returns:
        dict(str:int): 各渠道保量次数
    """
    channel_guaranteed = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    
    channel_guaranteed['com'] = max(math.ceil(com_login_cnt_last_month),0)
    channel_guaranteed['free'] = max(math.ceil(free_login_cnt_last_month),0)
    channel_guaranteed['strong_ctrl'] = max(math.ceil(strong_ctrl_login_cnt_last_month),0)
    channel_guaranteed['normal_ctrl'] = max(math.ceil(normal_ctrl_login_cnt_last_month),0)
    channel_guaranteed['no_ctrl'] = max(math.ceil(no_ctrl_login_cnt_last_month),0)
    
    return channel_guaranteed

def BusinessRuleSupplement(basic_result, user_label,  year_clv_threshold, gmv_year, clv, month_clv_threshold, 
                           com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                           normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month, 
                           my_hash_code,month_hash):
    """根据用户逻辑进行方案调整

    """
    channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
    businessrule_no_allocation_channel = []
    businessrule_plan = basic_result
    channel_businessrule_max = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    channel_businessrule_max_add = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    
   
    year_clv_threshold = 600.0

    if user_label == 'A1':
        # 根据业务逻辑调整渠道上限
        com_adjust_pra = 1.2
        hash_precent = 99
        msg_add = 1
    
        # 京东可控置0 
        businessrule_no_allocation_channel.append('normal_ctrl')

        # 商业化渠道补量
        # A1人群的商业化补量不超过20%
        # 商业化在上月基础上调整1.2倍的抽样分配结果
        com_adjust_day = sampling_supplement(channel_days = com_login_cnt_last_month,
                                             adjust_day = com_adjust_pra * com_login_cnt_last_month, # 1.2
                                             my_hash_code = my_hash_code)
        
        channel_businessrule_max['com'] = com_adjust_day  # A1人群的商业化补量不超过20%
   
        # 免费渠道补量
        businessrule_plan['free'] +=  1 
        channel_businessrule_max_add['free'] +=1
        
        # # 短信渠道补量
        # if (my_hash_code <= hash_precent):
        #     businessrule_plan['strong_ctrl'] += msg_add
        #     channel_businessrule_max_add['strong_ctrl'] += msg_add     
        # else:
        #     pass
       
    elif user_label == 'A2':
        if clv <= month_clv_threshold:
            businessrule_plan = basic_result
            
            # clv<50, 如果京东可控保量=0 则按月维度抽样（50%+1）
            if businessrule_plan['normal_ctrl'] == 0  and month_hash < 50:
                businessrule_plan['normal_ctrl'] +=1
                channel_businessrule_max_add['normal_ctrl'] += 1
        else:
            pass

    elif user_label == 'B':
        # 返回基础方案 所有渠道不分配
        businessrule_plan =  basic_result 
        businessrule_no_allocation_channel += channel_type      

    elif user_label == 'C':
        if gmv_year <= year_clv_threshold:
            # 返回基础方案 所有渠道不分配
            businessrule_plan = basic_result
            businessrule_no_allocation_channel += channel_type 
        else:
            businessrule_plan['normal_ctrl'] = min(20, 1.5 * normal_ctrl_login_cnt_last_month, businessrule_plan['normal_ctrl'])
    
    return businessrule_plan, businessrule_no_allocation_channel, channel_businessrule_max, channel_businessrule_max_add

def sampling_supplement(channel_days,adjust_day,my_hash_code):
    """按随机抽样的方法对渠道天数进行修正，
       举例说明: 
       假设该渠道有100人, 渠道干预次数默认值为2, 总干预次数为200
       整体干预次数要提升1.2倍即240, 则每个人的渠道干预次数为2.4, 
       干预次数按照抽样方式调整, 所有人干预次数调整为2, 40%的人在2的基础上再增加1次, 保证整体的干预次数是调整了1.2倍

    Args:
        channel_days (_type_): 渠道干预天数
        adjust_day (double): 需要调整的天数
        my_hash_code (string): 用户hash值, 用于抽样
        base_flag (_type_): _description_

    Returns:
        _type_: 调整后的渠道干预天数
    """

    adjust_day = adjust_day * 100 # 2.4
    
    adjust_day_base , adjust_day_hash_add = adjust_day//100 , int(adjust_day%100)
    
    # print(adjust_day_base , adjust_day_hash_add)
    channel_days = adjust_day_base # 2
    
    if my_hash_code <= adjust_day_hash_add: # 0.4
        channel_days += 1
    
    return channel_days

def get_target_allocation_login(part_dt,lift_login, natura_login, current_login,no_ctrl_login):
    """获取目标登录次数

    Args:
        lift_login (_type_): 阶跃所需引流次数
        natura_login (_type_): 本月自然预测登录次数
        current_login (_type_): 当前登录次数
        no_ctrl_login: 不可控渠道本月登录次数

    Returns:
        _type_: _description_
    """
    # part_dt = '2022-03-31'
    # TODO 当前登录次数要用各渠道*质量归一后求和数据，当前使用的是事实数据
    today =datetime.datetime.strptime(part_dt,'%Y-%m-%d') + timedelta(days=1)
    if today.day==1: 
        # 每月1号方案需要对当前登录次数置零
        current_login = min(current_login,0)
    else:
        pass
    
    return lift_login - current_login - natura_login - no_ctrl_login

def get_user_channel_max_func(priority_type,my_hash_code, channel_businessrule_max_add, channel_businessrule_max, 
                    com_max_login_count, free_max_login_count, strong_ctrl_max_login_count, normal_ctrl_max_login_count,no_ctrl_max_login_count,
                    com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month, normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month,
                    is_acce_by_com, is_acce_by_free, is_acce_by_strong_ctrl, is_acce_by_normal_ctrl, is_acce_by_no_ctrl,
                    month_clv_threshold,channel_max_adjust,channel_preference):
    """返回用户各渠道上下限
    Args:
        priority_type (str): 用户标签
        com_max_login_count (_type_): 商业化渠道上限(格子)
        free_max_login_count (_type_): 免费渠道上限(格子)
        strong_ctrl_max_login_count (_type_):强控渠道上限(格子)
        normal_ctrl_max_login_count (_type_): 可控渠道上限(格子)
        no_ctrl_max_login_count (_type_):不可控渠道上限(格子)
        com_login_cnt_last_month (_type_):商业化渠道上月干预次数(用户)
        free_login_cnt_last_month (_type_): 免费渠道上月干预次数(用户)
        strong_ctrl_login_cnt_last_month (_type_): 强控渠道上月干预次数(用户)
        normal_ctrl_login_cnt_last_month (_type_):可控渠道上月干预次数(用户)
        no_ctrl_login_cnt_last_month (_type_): 不可控渠道上月干预次数(用户)
        is_acce_by_com (bool): 商业化是否可达
        is_acce_by_free (bool): 免费是否可达
        is_acce_by_strong_ctrl (bool): 强控是否可达
        is_acce_by_normal_ctrl (bool): 可控是否可达
        is_acce_by_no_ctrl (bool): 不可控是否可达
        is_guaranteed_by_com (bool): 商业化保量次数
        is_guaranteed_by_free (bool): _description_
        is_guaranteed_by_strong_ctrl (bool): _description_
        is_guaranteed_by_normal_ctrl (bool): _description_
        is_guaranteed_by_no_ctrl (bool): _description_
        month_clv_threshold (_type_): a人群划分
        channel_max_adjust (_type_): 方案档位调节系数
        channel_preference (_type_): _description_

    Returns:
        channel_max_login_days(dict): 用户各渠道上限
        channel_min_login_days(dict): 用户各渠道下限
    """
    
    # channel_type = ['com','free','strong_ctrl','normal_ctrl','no_ctrl']
    # 获得channel_max_login_days
    is_acce_by_com= is_acce_by_com if is_acce_by_com else 1
    is_acce_by_free= is_acce_by_free if is_acce_by_free else 1
    is_acce_by_strong_ctrl= is_acce_by_strong_ctrl if is_acce_by_strong_ctrl else 1
    is_acce_by_normal_ctrl= is_acce_by_normal_ctrl if is_acce_by_normal_ctrl else 1
    is_acce_by_no_ctrl= is_acce_by_no_ctrl if is_acce_by_no_ctrl else 1

    channel_max_login_days = {'com' :is_acce_by_com,
                              'free':is_acce_by_free,
                              'strong_ctrl':is_acce_by_strong_ctrl,
                              'normal_ctrl':is_acce_by_normal_ctrl,
                              'no_ctrl':is_acce_by_no_ctrl}
    # 参数2 channel_max_adjust
    if channel_max_adjust == 999:
        channel_max_login_days['com'] = math.ceil(com_max_login_count) if is_acce_by_com >0 else channel_max_login_days['com'] 
        channel_max_login_days['free'] = math.ceil(free_max_login_count) if is_acce_by_free >0 else channel_max_login_days['free'] 
        channel_max_login_days['strong_ctrl'] = math.ceil(strong_ctrl_max_login_count) if is_acce_by_strong_ctrl >0 else channel_max_login_days['strong_ctrl'] 
        channel_max_login_days['normal_ctrl'] = math.ceil(normal_ctrl_max_login_count) if is_acce_by_normal_ctrl >0 else channel_max_login_days['normal_ctrl'] 
        
    else:
        # 上限修正:  1.2*上月>上限， 取上限； 1.2*上月<=上限，取1.2*上月
        # 随机抽样法调整渠道上限
        if (com_login_cnt_last_month * channel_max_adjust) > com_max_login_count:
            channel_max_login_days['com'] = com_max_login_count
        else:
            channel_max_login_days['com'] = sampling_supplement(channel_days = channel_max_login_days['com'],
                                                                adjust_day = com_login_cnt_last_month * channel_max_adjust,
                                                                my_hash_code = my_hash_code)
        # free 上限修正:  1.2*上月>上限，  取上限； 1.2*上月<=上限，取1.2*上月
        if (free_login_cnt_last_month * channel_max_adjust) > free_max_login_count:
            channel_max_login_days['free'] = free_max_login_count
        else:    
            channel_max_login_days['free'] = sampling_supplement(channel_days = channel_max_login_days['free'] , 
                                                             adjust_day = free_login_cnt_last_month * channel_max_adjust,
                                                             my_hash_code = my_hash_code)
        # strong 上限修正:  1.2*上月>上限，  取上限； 1.2*上月<=上限，取1.2*上月    
        if (strong_ctrl_login_cnt_last_month * channel_max_adjust) > strong_ctrl_max_login_count:
            channel_max_login_days['strong_ctrl'] = strong_ctrl_max_login_count
        else:      
            channel_max_login_days['strong_ctrl'] = sampling_supplement(channel_days = channel_max_login_days['strong_ctrl'],
                                                            adjust_day = strong_ctrl_login_cnt_last_month * channel_max_adjust,
                                                            my_hash_code = my_hash_code)
        # normal  上限修正:  1.2*上月>上限，取上限； 1.2*上月<=上限，取1.2*上月  
        if (normal_ctrl_login_cnt_last_month * channel_max_adjust) > normal_ctrl_max_login_count:
            channel_max_login_days['normal_ctrl'] = normal_ctrl_max_login_count
        else:    
            channel_max_login_days['normal_ctrl'] = sampling_supplement(channel_days = channel_max_login_days['normal_ctrl'],
                                                            adjust_day = normal_ctrl_login_cnt_last_month * channel_max_adjust,
                                                            my_hash_code = my_hash_code)
        
    channel_max_login_days['no_ctrl'] = math.ceil(1.0 * no_ctrl_login_cnt_last_month)
    
    # 按业务规则修正渠道上限 不超过业务逻辑上限
    # 业务规则加量则 渠道上限加量
    for channel in channel_max_login_days:
        channel_max_login_days[channel] += channel_businessrule_max_add[channel]
        channel_max_login_days[channel] = min(channel_max_login_days[channel], channel_businessrule_max[channel])


    # 获得channel_min_login_days
    # is_guaranteed_by_com = is_guaranteed_by_com if is_guaranteed_by_com else 1
    # is_guaranteed_by_free = is_guaranteed_by_free if is_guaranteed_by_free else 1
    # is_guaranteed_by_strong_ctrl = is_guaranteed_by_strong_ctrl if is_guaranteed_by_strong_ctrl else 0
    # is_guaranteed_by_normal_ctrl = is_guaranteed_by_normal_ctrl if is_guaranteed_by_normal_ctrl else 0
    # is_guaranteed_by_no_ctrl = is_guaranteed_by_no_ctrl if is_guaranteed_by_no_ctrl else 0

    # channel_min_login_days = {'com' : is_guaranteed_by_com,
    #                           'free' : is_guaranteed_by_free,
    #                           'strong_ctrl' : is_guaranteed_by_strong_ctrl,
    #                           'normal_ctrl' : is_guaranteed_by_normal_ctrl,
    #                           'no_ctrl' : is_guaranteed_by_no_ctrl} 
    # # 最大最小修正
    # # TODO verify this
    # for c_name in channel_max_login_days:
    #     channel_max_day,channel_min_day = channel_max_login_days[c_name] , channel_min_login_days[c_name]
    #     channel_max_login_days[c_name], channel_min_login_days[c_name] = max([channel_max_day, channel_min_day]), min([channel_max_day,channel_min_day])

    return channel_max_login_days

def get_channel_login_costs(channel_type,
                            com_login_cost,free_login_cost,strong_ctrl_login_cost,normal_login_cost,no_ctrl_login_cost,
                            com_login_cnt_per_dau,free_login_cnt_per_dau,strong_ctrl_login_cnt_per_dau,normal_ctrl_login_cnt_per_dau,no_ctrl_login_cnt_per_dau,
                            com_quality_coeff,free_quality_coeff,strong_ctrl_quality_coeff,normal_quality_coeff,no_ctrl_quality_coeff):
    """  返回渠道成本

    Args:
        channel_type (_type_): _description_
        com_login_cost (_type_): _description_
        free_login_cost (_type_): _description_
        strong_ctrl_login_cost (_type_): _description_
        normal_login_cost (_type_): _description_
        no_ctrl_login_cost (_type_): _description_
        com_login_cnt_per_dau (_type_): _description_
        free_login_cnt_per_dau (_type_): _description_
        strong_ctrl_login_cnt_per_dau (_type_): _description_
        normal_ctrl_login_cnt_per_dau (_type_): _description_
        no_ctrl_login_cnt_per_dau (_type_): _description_
        com_quality_coeff (_type_): _description_
        free_quality_coeff (_type_): _description_
        strong_ctrl_quality_coeff (_type_): _description_
        normal_quality_coeff (_type_): _description_
        no_ctrl_quality_coeff (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    channel_cost = [com_login_cost,free_login_cost,strong_ctrl_login_cost,normal_login_cost,no_ctrl_login_cost]
    channel_pre_dau = [com_login_cnt_per_dau,free_login_cnt_per_dau,strong_ctrl_login_cnt_per_dau,normal_ctrl_login_cnt_per_dau,no_ctrl_login_cnt_per_dau,]
    channel_coeff = [com_quality_coeff,free_quality_coeff,strong_ctrl_quality_coeff,normal_quality_coeff,no_ctrl_quality_coeff]

    dau_cost =[]
    for i in range(len(channel_cost)):
        if (channel_pre_dau[i] > 0 ): 
            # check
            cost = channel_cost[i] / channel_pre_dau[i] / channel_coeff[i]
        else:
            cost = channel_cost[i]
        dau_cost.append(cost)

    return dict(zip(channel_type, dau_cost))


def GreedySolver(target_login_days, businessrule_no_allocation_channel, businessrule_result, 
                channel_max_login_days,  channel_login_costs):

    # channel_type = ['com','free','strong_ctrl','normal_ctrl','no_ctrl']、
    # 业务规则置零渠道  
    no_allocation_channel_list = businessrule_no_allocation_channel
    no_allocation_channel_list.append('no_ctrl')
    # 业务规则输出的方案结果
    greedy_plan = businessrule_result

    login_costs_dict = channel_login_costs
    sorted_costs = sorted(login_costs_dict.items(), key=lambda kv: (kv[1], kv[0]))

    for i in range(len(sorted_costs)):    
        
        used_day = 0
        index = sorted_costs[i][0]
        # 被分配的次数 = 目标次数 - 渠道已分配的次数（保量+业务补量）
        should_target_login_cnt = target_login_days - businessrule_result[index]
        
        # 渠道不可分配，则跳过
        if index in no_allocation_channel_list:
            continue
        
        # 渠道上限修正为 渠道上限 - 已分配次数
        channel_max_day = math.ceil(channel_max_login_days[index])
        channel_max_day = channel_max_day - businessrule_result[index]

        used_day = min(channel_max_day, should_target_login_cnt)

        target_login_days -= used_day
        target_login_days = max(0, target_login_days)

        greedy_plan[index] += used_day

    return greedy_plan

def get_lastday_data_no_allocation(part_dp, part_dt):
    
    used_sql = """
        SELECT
            user_log_acct,
            user_life_cycle_type_1st,
            model_a_1st,
            model_b_1st,
            goal_group_1st,
            annual_clv_1st,
            grid_name_1st,
            clv_pred_1m,
            model_a,
            model_b,
            model_c,
            model_l,
            user_life_cycle_type,
            login_cnt_mtd,
            natural_login_cnt_mtd,
            com_login_cnt_mtd,
            free_login_cnt_mtd,
            strong_ctrl_login_cnt_mtd,
            normal_ctrl_login_cnt_mtd,
            no_ctrl_login_cnt_mtd,
            com_user_last_month_login_cnt,
            free_user_last_month_login_cnt,
            strong_ctrl_user_last_month_login_cnt,
            normal_ctrl_user_last_month_login_cnt,
            no_ctrl_user_last_month_login_cnt,
            com_user_base_guarteed_cnt,
            free_user_base_guarteed_cnt,
            strong_ctrl_user_base_guarteed_cnt,
            normal_ctrl_user_base_guarteed_cnt,
            no_ctrl_user_base_guarteed_cnt,
            com_user_acce_flag,
            free_user_acce_flag,
            strong_ctrl_user_acce_flag,
            normal_ctrl_user_acce_flag,
            no_ctrl_user_acce_flag,
            com_grid_max_login_cnt,
            free_grid_max_login_cnt,
            strong_ctrl_grid_max_login_cnt,
            normal_ctrl_grid_max_login_cnt,
            no_ctrl_grid_max_login_cnt,
            com_grid_last_month_total_cnt,
            free_grid_last_month_total_cnt,
            strong_ctrl_grid_last_month_total_cnt,
            normal_ctrl_grid_last_month_total_cnt,
            no_ctrl_grid_last_month_total_cnt,
            com_grid_quality_coeff,
            free_grid_quality_coeff,
            strong_ctrl_grid_quality_coeff,
            normal_ctrl_grid_quality_coeff,
            no_ctrl_grid_quality_coeff,
            com_grid_login_cnt_per_dau,
            free_grid_login_cnt_per_dau,
            strong_ctrl_grid_login_cnt_per_dau,
            normal_grid_login_cnt_per_dau,
            no_ctrl_grid_login_cnt_per_dau,
            com_grid_last_login_cost,
            free_grid_last_login_cost,
            strong_ctrl_grid_last_login_cost,
            normal_ctrl_grid_last_login_cost,
            no_ctrl_grid_last_login_cost,
            algo_total_info,
            natural_pred_login_cnt_rest,
            case when (com_login_cnt - com_login_cnt_mtd)>=0 then (com_login_cnt - com_login_cnt_mtd)
                else 0 end as com_login_cnt,
            case when (free_login_cnt - free_login_cnt_mtd)>=0 then (free_login_cnt - free_login_cnt_mtd)
                else 0 end as free_login_cnt,
            case when (strong_ctrl_login_cnt - strong_ctrl_login_cnt_mtd)>=0 then (strong_ctrl_login_cnt - strong_ctrl_login_cnt_mtd)
                else 0 end as strong_ctrl_login_cnt,
            case when (normal_ctrl_login_cnt - normal_ctrl_login_cnt_mtd)>=0 then (normal_ctrl_login_cnt - normal_ctrl_login_cnt_mtd)
                else 0 end as normal_ctrl_login_cnt,
            case when (no_ctrl_login_cnt - no_ctrl_login_cnt_mtd)>=0 then (no_ctrl_login_cnt - no_ctrl_login_cnt_mtd)
                else 0 end as no_ctrl_login_cnt,
            priority_level,
            pred_com_dau,
            pred_free_dau,
            pred_strong_ctrl_dau,
            pred_normal_ctrl_dau,
            pred_no_ctrl_dau,
            total_login_dau_ratio,
            pred_total_dau,
            case when (com_login_cnt + free_login_cnt + strong_ctrl_login_cnt + normal_ctrl_login_cnt + no_ctrl_login_cnt)>= (lower_lift_login_cnt-login_cnt_mtd) then 1
                else 0 end as is_solvable,
            end_time,
            is_malice_user,
            pred_natural_dau,
            priority_type
        FROM 
            app.app_yhzz_umc_algo_pin_interim
        where 
            dt = '{part_dt}'
            and dp ='{part_dp}'
        )interim 
        left join
        
        (
        select
        -- 目标登录次数
        grid_name_1st,
            COALESCE(lower_lift_login_cnt,0) as lower_lift_login_cnt

        FROM
            app.app_yhzz_umc_unit_grid
        WHERE
            dt = '{part_dt}'
            AND grid_name_1st is not null
            AND is_grid_valid = 1
        )grid on interim.grid_name_1st  = grid.grid_name_1st

    """.format(part_dt = part_dt, part_dp = part_dp)
    
    print("get lastday data sql", used_sql)
    data = spark.sql(used_sql)
    data.cache()
    print("data columns", data.columns)
    print("data dtypes", data.dtypes)

    return data

'''
def get_base_plan(result, com_login_cnt_last_month,free_login_cnt_last_month,
                  strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_mont):
    """用户兜底方案

    Args:
        result (_type_): _description_
        com_login_cnt_last_month (_type_): _description_
        free_login_cnt_last_month (_type_): _description_
        strong_ctrl_login_cnt_last_month (_type_): _description_
        normal_ctrl_login_cnt_last_month (_type_): _description_
        no_ctrl_login_cnt_last_mont (_type_): _description_

    Returns:
        dict: 用户各渠道兜底干预次数
    """
    # greedy_result = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
    base_result = result
    base_result['com'] = math.ceil(com_login_cnt_last_month) # 向上取整
    base_result['free'] = math.ceil(free_login_cnt_last_month) # 向上取整
    base_result['strong_ctrl'] = math.ceil(strong_ctrl_login_cnt_last_month) # 向上取整
    base_result['normal_ctrl'] = math.ceil(normal_ctrl_login_cnt_last_month) # 向上取整
    base_result['no_ctrl'] = math.ceil(no_ctrl_login_cnt_last_mont) # 向上取整

    return base_result
'''

if __name__ == '__main__':
    # 获取基础参数
    day_ago_1 =  sys.argv[1] # T-1
    dp = sys.argv[2] #  [low,medium,high,max]
    now = datetime.datetime.strptime(day_ago_1, '%Y-%m-%d')
    day_ago_1 =  datetime.datetime.strptime(day_ago_1, '%Y-%m-%d').strftime('%Y-%m-%d')
    now_month = now.month
    this_month_end = datetime.datetime.strftime(datetime.datetime(now.year, now.month + 1, 1) - timedelta(seconds=1),
                                                '%Y-%m-%d %H:%M:%S')
    table_name = 'app.app_yhzz_umc_algo_pin_interim'
    
    # 根据方案档位，获取方案参数
    print(">"*10,"get plan pra")
    a1_threshold = 30
    month_clv_threshold,  year_clv_threshold, channel_max_adjust, channel_preference = get_plan_pra(plan_level = dp)

    print('-----model params-----')
    print('day_ago_1(table_dt):   ', day_ago_1)
    print('this_month_end:        ', this_month_end)
    print('dp(table_dp):          ', dp)
    print('table_name:            ', table_name)
    print('month_clv_threshold:   ', month_clv_threshold)
    print('year_clv_threshold:    ', year_clv_threshold)
    print('channel_max_adjust:    ', channel_max_adjust)
    print('channel_preference:    ', channel_preference)
    print('a1_threshold:          ', a1_threshold)

    
    # 获取base数据
    print(">"*10,"get base data")
    base_data = get_base_data(part_dt = day_ago_1,
                              part_month= now_month)

    # 存储参数结果
    base_data = base_data.withColumn('month_clv_threshold', lit(month_clv_threshold))
    base_data = base_data.withColumn('year_clv_threshold', lit(year_clv_threshold))
    base_data = base_data.withColumn('channel_max_adjust', lit(channel_max_adjust))
    base_data = base_data.withColumn('channel_preference', array([lit(x) for x in channel_preference]))
    base_data = base_data.withColumn('a1_threshold', lit(a1_threshold))


    # 进行分配
    print(">"*10,"根据日期判断分配方式 进行流量分配") 
    base_data = juge_how_allocation(run_day = day_ago_1,
                                    base_data = base_data)
    

    print(">"*10,"数据处理:新增渠道引流登端结果数据列等")
    base_data = add_channel_result_col(data = base_data,
                                       this_month_end = this_month_end)    
    
    print(">"*10,"基础校验")
    # todo 基础数据校验
    print(">"*10,"写表")
    
    data_insert_table(data = base_data,
                      table_name = table_name,
                      table_dt = day_ago_1,
                      tabel_dp = dp)    
    
    # current_time放在程序的末尾
    current_time = time.time()
    print("运行时间为" + str(current_time - old_time) + "s")                                                                                                                                                                                                                                                                           

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-a4be3cc8c6e7>", line 206, in <module>
    @udf(returnType=MapType(StringType(), IntegerType()))
NameError: name 'udf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2016, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/us

NameError: name 'udf' is not defined

In [5]:
## 重构函数

@udf(returnType=MapType(StringType(), IntegerType()))
def get_result(part_dt,month_clv_threshold,year_clv_threshold,channel_max_adjust,channel_preference,a1_threshold,priority_type,my_hash_code,
                lower_lift_login_cnt, natural_pred_login_cnt_rest,login_cnt_mtd,natural_login_cnt_mtd,no_ctrl_login_cnt_mtd,
                is_acce_by_com, is_acce_by_free, is_acce_by_strong_ctrl, is_acce_by_normal_ctrl,is_acce_by_no_ctrl,
                com_max_login_count, free_max_login_count, strong_ctrl_max_login_count,normal_ctrl_max_login_count,no_ctrl_max_login_count,
                is_guaranteed_by_com, is_guaranteed_by_free, is_guaranteed_by_strong_ctrl,is_guaranteed_by_normal_ctrl, 
                is_guaranteed_by_no_ctrl,
                com_login_cost, free_login_cost, strong_ctrl_login_cost, normal_login_cost,no_ctrl_login_cost, 
                com_quality_coeff,free_quality_coeff,strong_ctrl_quality_coeff,normal_ctrl_quality_coeff,no_ctrl_quality_coeff,
                com_login_cnt_per_dau, free_login_cnt_per_dau,  strong_ctrl_login_cnt_per_dau, normal_ctrl_login_cnt_per_dau, no_ctrl_login_cnt_per_dau,
                clv_pred_1m,gmv_year,
                login_cnt_last_month,
                natural_login_cnt_last_month,com_login_cnt_last_month,free_login_cnt_last_month,strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month,
                com_login_cnt_mtd,free_login_cnt_mtd,strong_ctrl_login_cnt_mtd,normal_ctrl_login_cnt_mtd,month_hash):
# 修正3 处理渠道上限 减掉各渠道mtd值
    channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']

    # 目标
    print('target') # part_dt,lift_login, natura_login, current_login,no_ctrl_login
    target_login_days = get_target_allocation_login(part_dt = part_dt,
                                              lift_login =lower_lift_login_cnt,
                                              natura_login = (natural_login_cnt_last_month - natural_login_cnt_mtd), #用上月自然登录次数替代本月预测v4
                                              current_login = login_cnt_mtd,
                                              no_ctrl_login = (no_ctrl_login_cnt_last_month-no_ctrl_login_cnt_mtd))
    # 渠道上下限 & 参数调整
    channel_max_login_days, channel_min_login_days  = get_channel_max_and_min_func(priority_type,my_hash_code,
                                                                                    com_max_login_count, free_max_login_count, strong_ctrl_max_login_count, normal_ctrl_max_login_count,no_ctrl_max_login_count,
                                                                                    com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month, normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month,
                                                                                    is_acce_by_com, is_acce_by_free, is_acce_by_strong_ctrl, is_acce_by_normal_ctrl, is_acce_by_no_ctrl,
                                                                                    is_guaranteed_by_com, is_guaranteed_by_free, is_guaranteed_by_strong_ctrl,is_guaranteed_by_normal_ctrl, is_guaranteed_by_no_ctrl,
                                                                                    month_clv_threshold,channel_max_adjust,channel_preference)

    # 渠道成本
    channel_login_costs = get_channel_login_costs(channel_type,
                                                  com_login_cost, free_login_cost, strong_ctrl_login_cost,
                                                  normal_login_cost, no_ctrl_login_cost,
                                                  com_login_cnt_per_dau, free_login_cnt_per_dau,
                                                  strong_ctrl_login_cnt_per_dau, normal_ctrl_login_cnt_per_dau,
                                                  no_ctrl_login_cnt_per_dau,
                                                  com_quality_coeff, free_quality_coeff, strong_ctrl_quality_coeff,
                                                  normal_ctrl_quality_coeff, no_ctrl_quality_coeff)
    
    # 用户价值
    clv = get_user_clv(clv_pred_1m)

    # 贪心优化算法输出贪心解
    greedy_result = greedy_solver(target_login_days,
                                channel_max_login_days,
                                channel_min_login_days,
                                channel_login_costs,
                                clv)

    # 根据用户类型修正渠道分配方案
    gmv_year = gmv_year if gmv_year else 0
    user_label = priority_type
    
    final_result = check_user_plan(greedy_result, user_label, target_login_days, year_clv_threshold, gmv_year,clv,
                                   month_clv_threshold, a1_threshold,channel_max_adjust, 
                                   com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                                   normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month,
                                   login_cnt_last_month,
                                   com_login_cnt_mtd,free_login_cnt_mtd,
                                    strong_ctrl_login_cnt_mtd,normal_ctrl_login_cnt_mtd,no_ctrl_login_cnt_mtd,month_hash)
    

    # 结果最终修正1 不小于上月用户次数  
    final_result['com'] = math.ceil(max(final_result['com'],com_login_cnt_last_month))
    final_result['free'] = math.ceil(max(final_result['free'],free_login_cnt_last_month))
    final_result['strong_ctrl'] = math.ceil(max(final_result['strong_ctrl'],strong_ctrl_login_cnt_last_month))
    final_result['no_ctrl'] = math.ceil(max(final_result['no_ctrl'],no_ctrl_login_cnt_last_month))
    
    # 修正3 处理渠道上限 减掉各渠道mtd值
    # 结果最终修正3 减去渠道已完成
    final_result['com'] = math.ceil(max(final_result['com'] - com_login_cnt_mtd, 1))
    final_result['free'] = math.ceil(max(final_result['free'] - free_login_cnt_mtd, 0))
    final_result['strong_ctrl'] = math.ceil(max(final_result['strong_ctrl'] - strong_ctrl_login_cnt_mtd, 0))
    final_result['no_ctrl'] = math.ceil(max(final_result['no_ctrl'] - no_ctrl_login_cnt_mtd, 0))
    


    re = sum(list(final_result.values()))
    target_flag = 1 if re >= target_login_days else 0
    final_result['flag'] = target_flag

    
    return final_result

def check_user_plan(greedy_result, user_label, target_login_days, year_clv_threshold, gmv_year,clv,
                    month_clv_threshold, a1_threshold,channel_max_adjust,
                    com_login_cnt_last_month, free_login_cnt_last_month, strong_ctrl_login_cnt_last_month,
                    normal_ctrl_login_cnt_last_month, no_ctrl_login_cnt_last_month,
                    login_cnt_last_month,
                    com_login_cnt_mtd,free_login_cnt_mtd,
                    strong_ctrl_login_cnt_mtd,normal_ctrl_login_cnt_mtd,no_ctrl_login_cnt_mtd,month_hash):
    """根据用户逻辑进行方案调整

    Args:
        greedy_result (_type_): _description_
        user_label (_type_): _description_
        target_login_days (_type_): _description_
        year_clv_threshold (_type_): _description_
        annual_clv_1st (_type_): _description_
        clv (_type_): _description_
        month_clv_threshold (_type_): _description_
        a1_threshold (_type_): _description_
        channel_max_adjust (_type_): _description_
        com_login_cnt_last_month (_type_): _description_
        free_login_cnt_last_month (_type_): _description_
        strong_ctrl_login_cnt_last_month (_type_): _description_
        normal_ctrl_login_cnt_last_month (_type_): _description_
        no_ctrl_login_cnt_last_month (_type_): _description_
        natural_pred_login_cnt_n30d (_type_): _description_

    Returns:
        _type_: _description_
    """
    # channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
   
    final_result = greedy_result
    year_clv_threshold = 600.0
    login_cnt_last_month = login_cnt_last_month if login_cnt_last_month else 5
    a1_threshold = 10
    user_week_random_num = month_hash

    if user_label == 'A1':
        pass
        # if login_cnt_last_month< a1_threshold:
        #     final_result['normal_ctrl'] = math.ceil(normal_ctrl_login_cnt_last_month)
        # else:
        #     final_result['normal_ctrl'] = 0
        
        # final_result['com'] = min(int(1.2 * com_login_cnt_last_month),final_result['com'] )

    elif user_label == 'A2':
        if clv <= month_clv_threshold:
            # clv<50, 如果京东可控保量=0 则按月维度抽样（50%+1）
            final_result = get_base_plan(greedy_result, com_login_cnt_last_month,free_login_cnt_last_month,
                                         strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month)
            if final_result['normal_ctrl']==0 and user_week_random_num< 50:
                final_result['normal_ctrl']+=1
        else:
            pass

    elif user_label == 'B':
        final_result = get_base_plan(greedy_result, com_login_cnt_last_month,free_login_cnt_last_month,
                                         strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month)

    elif user_label == 'C':
        # TODO c人群修正
        if gmv_year <= year_clv_threshold:
            # 兜底
            final_result = get_base_plan(greedy_result, com_login_cnt_last_month,free_login_cnt_last_month,
                                         strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month)

        else:
            
            final_result['normal_ctrl'] = min(20, 1.5 * normal_ctrl_login_cnt_last_month,final_result['normal_ctrl'])
    
    ## 修正2 可控置零
    if user_label == 'A1':
        final_result['normal_ctrl'] = 0
    else:
        final_result['normal_ctrl'] = math.ceil(max(final_result['normal_ctrl'], normal_ctrl_login_cnt_last_month))
        final_result['normal_ctrl'] = math.ceil(max(final_result['normal_ctrl'] - normal_ctrl_login_cnt_mtd, 0))

    
    return final_result

In [6]:
# 获取基础参数

day_ago_1 =  '2022-05-01' # T-1
dp = 'lowv2' #  [low,medium,high,max]
now = datetime.datetime.strptime(day_ago_1, '%Y-%m-%d')
day_ago_1 =  datetime.datetime.strptime(day_ago_1, '%Y-%m-%d').strftime('%Y-%m-%d')
now_month = now.month
this_month_end = datetime.datetime.strftime(datetime.datetime(now.year, now.month + 1, 1) - timedelta(seconds=1),
                                            '%Y-%m-%d %H:%M:%S')
table_name = 'app.app_yhzz_umc_algo_pin_interim'

# 根据方案档位，获取方案参数
print(">"*10,"get plan pra")
a1_threshold = 30
month_clv_threshold,  year_clv_threshold, channel_max_adjust, channel_preference = get_plan_pra(plan_level = dp)

print('-----model params-----')
print('day_ago_1(table_dt):   ', day_ago_1)
print('this_month_end:        ', this_month_end)
print('dp(table_dp):          ', dp)
print('table_name:            ', table_name)
print('month_clv_threshold:   ', month_clv_threshold)
print('year_clv_threshold:    ', year_clv_threshold)
print('channel_max_adjust:    ', channel_max_adjust)
print('channel_preference:    ', channel_preference)
print('a1_threshold:          ', a1_threshold)




# 存储参数结果
base_data = base_data.withColumn('month_clv_threshold', lit(month_clv_threshold))
base_data = base_data.withColumn('year_clv_threshold', lit(year_clv_threshold))
base_data = base_data.withColumn('channel_max_adjust', lit(channel_max_adjust))
base_data = base_data.withColumn('channel_preference', array([lit(x) for x in channel_preference]))
base_data = base_data.withColumn('a1_threshold', lit(a1_threshold))

# 第一层分配(保量准则)
print(">"*10,"第一层分配 保量分配逻辑")
base_data = base_data.withColumn('guaranteed',get_channel_guaranteed_cnt('com_login_cnt_last_month',
                                                                            'free_login_cnt_last_month',
                                                                            'strong_ctrl_login_cnt_last_month',
                                                                            'normal_ctrl_login_cnt_last_month',
                                                                            'no_ctrl_login_cnt_last_month'))
# 引流次数
base_data = base_data.withColumn('is_guaranteed_by_com', base_data["guaranteed"].getItem("com"))
base_data = base_data.withColumn('is_guaranteed_by_free', base_data["guaranteed"].getItem("free"))
base_data = base_data.withColumn('is_guaranteed_by_strong_ctrl', base_data["guaranteed"].getItem("strong_ctrl"))
base_data = base_data.withColumn('is_guaranteed_by_normal_ctrl', base_data["guaranteed"].getItem("normal_ctrl"))
base_data = base_data.withColumn('is_guaranteed_by_no_ctrl', base_data["guaranteed"].getItem("no_ctrl"))

# 第二层分配(动态分配)
print(">"*10,"第二层分配 动态分配") 
base_data = base_data.withColumn('result',get_result('dt','month_clv_threshold','year_clv_threshold','channel_max_adjust',
                                                        'channel_preference','a1_threshold','priority_type','my_hash_code',
                                                        'lower_lift_login_cnt','natural_pred_login_cnt_rest','login_cnt_mtd','natural_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                                                        'is_acce_by_com','is_acce_by_free','is_acce_by_strong_ctrl',
                                                        'is_acce_by_normal_ctrl','is_acce_by_no_ctrl',
                                                        'com_max_login_count','free_max_login_count','strong_ctrl_max_login_count',
                                                        'normal_ctrl_max_login_count','no_ctrl_max_login_count',
                                                        'is_guaranteed_by_com','is_guaranteed_by_free','is_guaranteed_by_strong_ctrl',
                                                        'is_guaranteed_by_normal_ctrl','is_guaranteed_by_no_ctrl',
                                                        'com_login_cost','free_login_cost','strong_ctrl_login_cost','normal_login_cost',
                                                        'no_ctrl_login_cost',
                                                        'com_quality_coeff','free_quality_coeff','strong_ctrl_quality_coeff',
                                                        'normal_ctrl_quality_coeff','no_ctrl_quality_coeff',
                                                        'com_login_cnt_per_dau','free_login_cnt_per_dau',
                                                        'strong_ctrl_login_cnt_per_dau','normal_ctrl_login_cnt_per_dau',
                                                        'no_ctrl_login_cnt_per_dau',
                                                        'clv_pred_1m','gmv_year',
                                                        'login_cnt_last_month','natural_login_cnt_last_month','com_login_cnt_last_month','free_login_cnt_last_month','strong_ctrl_login_cnt_last_month','normal_ctrl_login_cnt_last_month','no_ctrl_login_cnt_last_month',
                                                        'com_login_cnt_mtd','free_login_cnt_mtd','strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','month_hash'))
                                                        # 修正3 处理渠道上限 减掉各渠道mtd值

print(">"*10,"二次调控") 
# TODO 二次调控代码

print(">"*10,"数据处理:新增渠道引流登端结果数据列等")
base_data = add_channel_result_col(data = base_data,
                                    this_month_end = this_month_end)    



>>>>>>>>>> get plan pra
-----model params-----
day_ago_1(table_dt):    2022-05-01
this_month_end:         2022-05-31 23:59:59
dp(table_dp):           lowv2
table_name:             app.app_yhzz_umc_algo_pin_interim
month_clv_threshold:    50
year_clv_threshold:     1
channel_max_adjust:     1.0
channel_preference:     [1.0, 1.0, 1.0, 1.0, 1.0]
a1_threshold:           30
>>>>>>>>>> 第一层分配 保量分配逻辑
>>>>>>>>>> 第二层分配 动态分配
>>>>>>>>>> 二次调控
>>>>>>>>>> 数据处理:新增渠道引流登端结果数据列等


In [7]:
base_data.createOrReplaceTempView('result_data_tmp')


sql = """

    --  代码重构验数
    SELECT
        priority_type, 
        sum(com_login_cnt) as com_login_cnt,
        sum(free_login_cnt) as free_login_cnt,
        sum(strong_ctrl_login_cnt) as strong_ctrl_login_cnt,
        sum(normal_ctrl_login_cnt) as normal_ctrl_login_cnt,
        sum(no_ctrl_login_cnt) as no_ctrl_login_cnt
    FROM
        result_data_tmp
    GROUP BY
    priority_type
    ORDER BY
    priority_type
"""
# print(sql)
result = spark.sql(sql)
result.show()



Py4JJavaError: An error occurred while calling o506.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 6 times, most recent failure: Lost task 0.5 in stage 3.0 (TID 1210, BJHTYD-Hope-11-11-21-4.hadoop.jd.local, executor 50): java.io.IOException: Cannot run program "/usr/local/anaconda3//bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:126)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:109)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.

In [87]:

class TrafficAllocation():
    def __init__(self,data):
        self.data = data
        
    @staticmethod    
    @udf(IntegerType())
    def BasicGuarantee(base_plan):
        base_plan = 1
        return base_plan
    
    @staticmethod
    @udf(IntegerType())
    def BusinessSupplement(self):
        yewu_plan = 1
        return yewu_plan
    
    @staticmethod
    @udf(IntegerType())
    def GreedySolver(val):
        val += 1
        return val
    
    def get_allocation_result(self):
        # 基础保量方案
        self.data = self.data.withColumn('test', self.BasicGuarantee(col('com_base_cnt')))
        
        # 补量
        # 动态调控
    

In [88]:
func = TrafficAllocation(test_data)
func.get_allocation_result()

In [1]:
import math

In [130]:
a1_threshold = 30
month_clv_threshold = 130
year_clv_threshold = 50 
channel_max_adjust=1.2 
channel_preference=[1,1,1,1,1]

class TrafficAllocation():
    """流量分配函数, 包含基础保量、业务补量以及贪心阶跃算法
    """
    def __init__(self,a1_threshold):
        self.a1_threshold = a1_threshold
    
    def get_target_allocation_login(self,part_dt,lift_login, natura_login, current_login,no_ctrl_login):
        
        today =datetime.datetime.strptime(part_dt,'%Y-%m-%d') + timedelta(days=1)
        if today.day==1: 
            # 每月1号方案需要对当前登录次数置零
            current_login = min(current_login,0)
        else:
            pass
    
        return lift_login - current_login - natura_login - no_ctrl_login
        
        
    def BasicGuarantee(self, user_lastmonth_channel_login):
        
        user_lastmonth_channel_login = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
        user_channel_basicguaranteed = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
        
        for channel_name in user_lastmonth_channel_login:
            print(channel_name)
            user_channel_basicguaranteed[channel_name] = max(math.ceil(user_lastmonth_channel_login[channel_name]),0)
        
        return user_channel_basicguaranteed
    
    def BusinessRuleSupplement(self, base_plan, user_type, my_hash_code, user_month_hash,
                                com_login_cnt_last_month, free_login_cnt_last_month, 
                                strong_ctrl_login_cnt_last_month, normal_ctrl_login_cnt_last_month, 
                                no_ctrl_login_cnt_last_month,
                                com_login_cnt_mtd, free_login_cnt_mtd, strong_ctrl_login_cnt_mtd, 
                                normal_ctrl_login_cnt_mtd, no_ctrl_login_cnt_mtd
                                user_month_clv):
        """业务补量有三个操作, 渠道置零、加量、不超过某上限
        """
        businessrule_no_allocation_channel = []
        businessrule_plan = base_plan
        channel_businessrule_max = {'com': 0, 'free': 0, 'strong_ctrl': 0, 'normal_ctrl': 0, 'no_ctrl': 0}
        user_lastmonth_channel_login = {'com': com_login_cnt_last_month, 
                                        'free': free_login_cnt_last_month, 
                                        'strong_ctrl': strong_ctrl_login_cnt_last_month, 
                                        'normal_ctrl': normal_ctrl_login_cnt_last_month, 
                                        'no_ctrl': no_ctrl_login_cnt_last_month}


        if user_type== 'A1':
            com_adjust_pra = 1.2
            hash_precent = 28
            msg_add = 1
        
            # 京东可控置0 
            businessrule_no_allocation_channel.append('normal_ctrl')
            # 商业化渠道补量#A1人群的商业化补量不超过20%
            # 商业化在上月基础上调整1.2倍的抽样分配结果 
            com_adjust_day = sampling_supplement(channel_days = user_lastmonth_channel_login['com'],
                                                 adjust_day = com_adjust_pra * user_lastmonth_channel_login['com'], # 1.2
                                                 my_hash_code = my_hash_code)
            # A1人群的商业化补量不超过20%
            channel_businessrule_max['com'] = com_adjust_day 
            # 免费渠道补量
            businessrule_plan['free'] +=1 
            # 短信渠道补量
            if (my_hash_code<= hash_precent):
                businessrule_plan['strong_ctrl'] += msg_add
            else:
                pass
        
        elif user_type=='A2':
            month_clv_threshold = 50 
            
            if user_month_clv <= month_clv_threshold:
                # clv<50, 如果京东可控保量=0 则按月维度抽样（50%+1）
                businessrule_plan = get_base_plan(businessrule_plan, com_login_cnt_last_month,free_login_cnt_last_month,
                                            strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month)
            # 可控为0则 50%概率+1    
            if businessrule_plan['normal_ctrl'] == 0 and user_month_hash < 50:
                    businessrule_plan['normal_ctrl'] += 1
            else:
                pass
            
        elif user_type =='B':
            businessrule_plan = get_base_plan(businessrule_plan, com_login_cnt_last_month,free_login_cnt_last_month,
                                         strong_ctrl_login_cnt_last_month,normal_ctrl_login_cnt_last_month,no_ctrl_login_cnt_last_month)
        
        elif user_type=='C':
            pass
        
        
        return businessrule_plan
 
    def GreedySolver(self):
        print(self.val)
        self.val += 1
        return int(self.val)
    
@udf(IntegerType())
def get_allocation_result(lower_lift_login_cnt,natural_login_cnt_last_month,no_ctrl_login_cnt_last_month,
                          login_cnt_mtd,natural_login_cnt_mtd, no_ctrl_login_cnt_mtd):
    
    func = TrafficAllocation(channel_max_adjust)
    
    # 基础保量方案
    basic_plan = func.BasicGuarantee()
    
    # 业务补量
    business_plan = func.BusinessSupplement(basic_plan)
    
    # 贪心阶跃
    greedy_plan = func.GreedySolver(business_plan)
    
    # 数据校验
    finnal_plan = greedy_plan
    
    return finnal_plan

In [1]:
def sampling_supplement(channel_days,adjust_day,my_hash_code):
    """按随机抽样的方法对渠道天数进行修正，
       举例说明: 
       假设该渠道有100人, 渠道干预次数默认值为2, 总干预次数为200
       整体干预次数要提升1.2倍即240, 则每个人的渠道干预次数为2.4, 
       干预次数按照抽样方式调整, 所有人干预次数调整为2, 40%的人在2的基础上再增加1次, 保证整体的干预次数是调整了1.2倍

    Args:
        channel_days (_type_): 渠道干预天数
        adjust_day (double): 需要调整的天数
        my_hash_code (string): 用户hash值, 用于抽样
        base_flag (_type_): _description_

    Returns:
        _type_: 调整后的渠道干预天数
    """

    adjust_day = adjust_day * 100 # 2.4
    
    adjust_day_base , adjust_day_hash_add = adjust_day//100 , int(adjust_day%100)
    
    # print(adjust_day_base , adjust_day_hash_add)
    channel_days = adjust_day_base # 2
    
    if my_hash_code <= adjust_day_hash_add: # 0.4
        channel_days += 1
    
    return channel_days

In [91]:
l =[(1,   10   ,  'F')
,(2 ,   2   ,  'M')
,(2 ,  10  ,   'F')
,(2 ,  3  ,    'F')
,(3 ,  10,     'M')]
columns = ['id',  'Age',  'Gender']
df=spark.createDataFrame(l, columns)
df.show(2)


+---+---+------+
| id|Age|Gender|
+---+---+------+
|  1| 10|     F|
|  2|  2|     M|
+---+---+------+
only showing top 2 rows



In [131]:
df = df.withColumn('test',get_allocation_result('id','Age'))

In [132]:
df.show(2)

+---+---+------+----+
| id|Age|Gender|test|
+---+---+------+----+
|  1| 10|     F|   2|
|  2|  2|     M|   2|
+---+---+------+----+
only showing top 2 rows



In [8]:
businessrule_no_allocation_channel = []
channel_type = ['com','free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']
businessrule_no_allocation_channel+=channel_type
businessrule_no_allocation_channel

['com', 'free', 'strong_ctrl', 'normal_ctrl', 'no_ctrl']

In [9]:
businessrule_no_allocation_channel.append('no_ctrl')

In [10]:
'no_ctrl' in businessrule_no_allocation_channel

True

## 二次调控梳理
T为首日,day为现在的时间
* 判断当前时间选择的调控方式（当前高活）
    * day = T ，直接跑首日分配逻辑
    * 高活 （day-T）是3的倍数 重跑首日分配
    * 高活  （day-T）== 7 渠道内调控

* 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-3ac120340dd4>", line 1, in <module>
    @udf(returnType=MapType(StringType(), IntegerType()))
NameError: name 'udf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2016, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/

NameError: name 'udf' is not defined

In [5]:
import datetime

In [10]:
run_day = '2022-05-16'
first_allocation_day = datetime.datetime.strptime('2022-05-15','%Y-%m-%d')
run_day = datetime.datetime.strptime(run_day,'%Y-%m-%d')
day1_ago_runday = datetime.datetime.strftime(run_day- timedelta(days=1),'%Y-%m-%d')
day1_ago_runday

'2022-05-15'

In [12]:
6%3 == 0

True

In [11]:
def juge_how_allocation(run_day, base_data):
    
    # TODO 5月临时方案 修改为整体方案
    first_allocation_day = datetime.datetime.strptime('2022-05-15','%Y-%M-%d')
    run_day = datetime.datetime.strptime(run_day,'%Y-%M-%d')
    day1_ago_runday = datetime.datetime.strftime(run_day- timedelta(days=1),'%Y-%M-%d')
    
    delta_day = (run_day-first_allocation_day).days
    
    if delta_day == 0:
        #  首日分配
        print(">"*10,"进行首日分配") 
        final_data = base_data.withColumn('result',get_allocation_result('dt','month_clv_threshold','year_clv_threshold','channel_max_adjust',
                                                         'channel_preference','a1_threshold','priority_type','my_hash_code',
                                                         'lower_lift_login_cnt','natural_pred_login_cnt_rest','login_cnt_mtd','natural_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                                                         'is_acce_by_com','is_acce_by_free','is_acce_by_strong_ctrl',
                                                         'is_acce_by_normal_ctrl','is_acce_by_no_ctrl',
                                                         'com_max_login_count','free_max_login_count','strong_ctrl_max_login_count',
                                                         'normal_ctrl_max_login_count','no_ctrl_max_login_count',
                                                         'com_login_cost','free_login_cost','strong_ctrl_login_cost','normal_login_cost',
                                                         'no_ctrl_login_cost',
                                                         'com_quality_coeff','free_quality_coeff','strong_ctrl_quality_coeff',
                                                         'normal_ctrl_quality_coeff','no_ctrl_quality_coeff',
                                                         'com_login_cnt_per_dau','free_login_cnt_per_dau',
                                                         'strong_ctrl_login_cnt_per_dau','normal_ctrl_login_cnt_per_dau',
                                                         'no_ctrl_login_cnt_per_dau',
                                                         'clv_pred_1m','gmv_year',
                                                         'login_cnt_last_month','natural_login_cnt_last_month','com_login_cnt_last_month','free_login_cnt_last_month','strong_ctrl_login_cnt_last_month','normal_ctrl_login_cnt_last_month','no_ctrl_login_cnt_last_month',
                                                         'com_login_cnt_mtd','free_login_cnt_mtd','strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','month_hash'))

    elif delta_day%3 == 0:
        # TODO 高活和低活的调整周期不一样 现在只写了高活
        # 3日-高活 渠道内调整 执行首日分配算法
        print(">"*10,"进行高活用户 渠道内调整") 
        final_data = base_data.withColumn('result',get_allocation_result('dt','month_clv_threshold','year_clv_threshold','channel_max_adjust',
                                                         'channel_preference','a1_threshold','priority_type','my_hash_code',
                                                         'lower_lift_login_cnt','natural_pred_login_cnt_rest','login_cnt_mtd','natural_login_cnt_mtd','no_ctrl_login_cnt_mtd',
                                                         'is_acce_by_com','is_acce_by_free','is_acce_by_strong_ctrl',
                                                         'is_acce_by_normal_ctrl','is_acce_by_no_ctrl',
                                                         'com_max_login_count','free_max_login_count','strong_ctrl_max_login_count',
                                                         'normal_ctrl_max_login_count','no_ctrl_max_login_count',
                                                         'com_login_cost','free_login_cost','strong_ctrl_login_cost','normal_login_cost',
                                                         'no_ctrl_login_cost',
                                                         'com_quality_coeff','free_quality_coeff','strong_ctrl_quality_coeff',
                                                         'normal_ctrl_quality_coeff','no_ctrl_quality_coeff',
                                                         'com_login_cnt_per_dau','free_login_cnt_per_dau',
                                                         'strong_ctrl_login_cnt_per_dau','normal_ctrl_login_cnt_per_dau',
                                                         'no_ctrl_login_cnt_per_dau',
                                                         'clv_pred_1m','gmv_year',
                                                         'login_cnt_last_month','natural_login_cnt_last_month','com_login_cnt_last_month','free_login_cnt_last_month','strong_ctrl_login_cnt_last_month','normal_ctrl_login_cnt_last_month','no_ctrl_login_cnt_last_month',
                                                         'com_login_cnt_mtd','free_login_cnt_mtd','strong_ctrl_login_cnt_mtd','normal_ctrl_login_cnt_mtd','month_hash'))
   
    elif (delta_day==7) or (delta_day-7)%6==0:
        # TODO 无渠道间调整
        # 7日 监控期，然后高活6d周期 渠道间调整
        print(">"*10,"进行高活用户 渠道间调整") 
        lastday_data = get_lastday_data_no_allocation(dp, day1_ago_runday)
        final_data = lastday_data
    
    elif delta_day%3 > 0:
        # 非首日 非调控期 取昨日分区减去昨日已完成
        print(">"*10,"非首日，非调控期，取昨日分区减去昨日已完成") 
        lastday_data = get_lastday_data_no_allocation(dp, day1_ago_runday)
        final_data = lastday_data
    
    return final_data
        
    

In [3]:

def get_lastday_data_no_allocation(part_dp, part_dt):
    
    used_sql = """
        SELECT
            user_log_acct,
            user_life_cycle_type_1st,
            model_a_1st,
            model_b_1st,
            goal_group_1st,
            annual_clv_1st,
            grid_name_1st,
            clv_pred_1m,
            model_a,
            model_b,
            model_c,
            model_l,
            user_life_cycle_type,
            login_cnt_mtd,
            natural_login_cnt_mtd,
            com_login_cnt_mtd,
            free_login_cnt_mtd,
            strong_ctrl_login_cnt_mtd,
            normal_ctrl_login_cnt_mtd,
            no_ctrl_login_cnt_mtd,
            com_user_last_month_login_cnt,
            free_user_last_month_login_cnt,
            strong_ctrl_user_last_month_login_cnt,
            normal_ctrl_user_last_month_login_cnt,
            no_ctrl_user_last_month_login_cnt,
            com_user_base_guarteed_cnt,
            free_user_base_guarteed_cnt,
            strong_ctrl_user_base_guarteed_cnt,
            normal_ctrl_user_base_guarteed_cnt,
            no_ctrl_user_base_guarteed_cnt,
            com_user_acce_flag,
            free_user_acce_flag,
            strong_ctrl_user_acce_flag,
            normal_ctrl_user_acce_flag,
            no_ctrl_user_acce_flag,
            com_grid_max_login_cnt,
            free_grid_max_login_cnt,
            strong_ctrl_grid_max_login_cnt,
            normal_ctrl_grid_max_login_cnt,
            no_ctrl_grid_max_login_cnt,
            com_grid_last_month_total_cnt,
            free_grid_last_month_total_cnt,
            strong_ctrl_grid_last_month_total_cnt,
            normal_ctrl_grid_last_month_total_cnt,
            no_ctrl_grid_last_month_total_cnt,
            com_grid_quality_coeff,
            free_grid_quality_coeff,
            strong_ctrl_grid_quality_coeff,
            normal_ctrl_grid_quality_coeff,
            no_ctrl_grid_quality_coeff,
            com_grid_login_cnt_per_dau,
            free_grid_login_cnt_per_dau,
            strong_ctrl_grid_login_cnt_per_dau,
            normal_grid_login_cnt_per_dau,
            no_ctrl_grid_login_cnt_per_dau,
            com_grid_last_login_cost,
            free_grid_last_login_cost,
            strong_ctrl_grid_last_login_cost,
            normal_ctrl_grid_last_login_cost,
            no_ctrl_grid_last_login_cost,
            algo_total_info,
            natural_pred_login_cnt_rest,
            case when (com_login_cnt - com_login_cnt_mtd)>=0 then (com_login_cnt - com_login_cnt_mtd)
                else 0 end as com_login_cnt,
            case when (free_login_cnt - free_login_cnt_mtd)>=0 then (free_login_cnt - free_login_cnt_mtd)
                else 0 end as free_login_cnt,
            case when (strong_ctrl_login_cnt - strong_ctrl_login_cnt_mtd)>=0 then (strong_ctrl_login_cnt - strong_ctrl_login_cnt_mtd)
                else 0 end as strong_ctrl_login_cnt,
            case when (normal_ctrl_login_cnt - normal_ctrl_login_cnt_mtd)>=0 then (normal_ctrl_login_cnt - normal_ctrl_login_cnt_mtd)
                else 0 end as normal_ctrl_login_cnt,
            case when (no_ctrl_login_cnt - no_ctrl_login_cnt_mtd)>=0 then (no_ctrl_login_cnt - no_ctrl_login_cnt_mtd)
                else 0 end as no_ctrl_login_cnt,
            priority_level,
            pred_com_dau,
            pred_free_dau,
            pred_strong_ctrl_dau,
            pred_normal_ctrl_dau,
            pred_no_ctrl_dau,
            total_login_dau_ratio,
            pred_total_dau,
            case when (com_login_cnt + free_login_cnt + strong_ctrl_login_cnt + normal_ctrl_login_cnt + no_ctrl_login_cnt)>= (lower_lift_login_cnt-login_cnt_mtd) then 1
                else 0 end as is_solvable,
            end_time,
            is_malice_user,
            pred_natural_dau,
            priority_type
        FROM 
            app.app_yhzz_umc_algo_pin_interim
        where 
            dt = '{part_dt}'
            and dp ='{part_dp}'
        )interim 
        left join
        
        (
        select
        -- 目标登录次数
        grid_name_1st,
            COALESCE(lower_lift_login_cnt,0) as lower_lift_login_cnt

        FROM
            app.app_yhzz_umc_unit_grid
        WHERE
            dt = '{part_dt}'
            AND grid_name_1st is not null
            AND is_grid_valid = 1
        )grid on interim.grid_name_1st  = grid.grid_name_1st

    """.format(part_dt = part_dt, part_dp = part_dp)
    
    print("get lastday data sql", used_sql)
    data = spark.sql(used_sql)
    data.cache()
    print("data columns", data.columns)
    print("data dtypes", data.dtypes)

    return data
    

In [34]:

run_month_firstday

'2022-06-01'

In [46]:

# 运行分区
run_day = '2022-06-01' 
# 分配计划执行时间 = 运行分区 + 1
plan_day = datetime.datetime.strptime(run_day,'%Y-%m-%d') - timedelta(days=- 1)
plan_year = plan_day.year
plan_month = plan_day.month

# 获取每月首日分配方案时间   
if plan_month== 5 and run_day < '2022-05-30':
    # 5月是05-15分区 ，05-16开始执行
    first_allocation_day = datetime.datetime.strptime('2022-05-16','%Y-%m-%d')
    
else:
    # 其他是每月第一天
    run_month_firstday = datetime.datetime.strftime(datetime.datetime(plan_year, plan_month, 1),'%Y-%m-%d')  
    first_allocation_day = datetime.datetime.strptime(run_month_firstday,'%Y-%m-%d')


day1_ago_runday = datetime.datetime.strftime(datetime.datetime.strptime(run_day,'%Y-%m-%d') - timedelta(days=1),'%Y-%m-%d')
delta_day = (plan_day -first_allocation_day).days
plan_flag = "none"


print("本月首日为", first_allocation_day)
print("分区为",run_day)
print("计划执行日期为",plan_day)
print("the delta_day is",delta_day)


本月首日为 2022-06-01 00:00:00
分区为 2022-06-01
计划执行日期为 2022-06-02 00:00:00
the delta_day is 1


In [21]:
(datetime.datetime.strptime(run_day,'%Y-%M-%d') -first_allocation_day).days

-113

In [24]:
datetime.datetime.strptime(run_day,'%Y-%m-%d')

datetime.datetime(2022, 5, 22, 0, 0)

In [12]:
def channelcols_to_channeldict(col1, col2, col3, col4, col5):
    
    cols_list = [col1, col2, col3, col4, col5]
    cols_dicts = dict(zip(channel_type,cols_list))
    
    return cols_dicts

In [13]:
channel_type = ["com", "free", "strong_ctrl", "normal_ctrl", "no_ctrl"]
channelcols_to_channeldict(channel_type, 1 ,2,3,4,5)

{'com': 1, 'free': 2, 'strong_ctrl': 3, 'normal_ctrl': 4, 'no_ctrl': 5}

In [11]:

with open(sql_file, 'r') as f:
    used_sql = f.read()

In [3]:
global_var1 = 100
@udf
def f1(x1):
    return x1 + global_var1

NameError: name 'udf' is not defined

In [6]:
test = {'free':0}
businessrule_plan = test
businessrule_plan['free'] +=  1 
businessrule_plan

{'free': 1}

In [61]:
day_plan_list = np.zeros((31, 6))
day_plan_list[:,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [63]:
day_select_natural =np.ones((1, 30))
day_select_natural

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [62]:
idx_natural = np.stack(day_select_natural.astype('int64'))
idx_natural-1

array([26, -1, 23, 13,  5, 27, 25,  4,  7,  3, 22, 17, 21, 16, 19, 14, 15,
       11,  1,  2, 20,  0,  8,  9, 28, 10, 12, 24,  6, 29])

In [50]:
day_plan_list[:,0]
## 次数是31次的结果为什么只有第一条是1



array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [58]:
cum_probability_2=np.array([[0.10335037,0.03985507,0.17425432,0.,0.07294118,0.138833]
                   ,[0.18270869,0.07246377,0.27629513,0.,0.13176471,0.17303823]
,[0.2423339,0.09299517,0.34850863,0.66666667,0.17529412,0.19315895]
,[0.29386712,0.12801932,0.42543171,0.66666667,0.21882353,0.21830986]
,[0.33333333,0.14009662,0.44270016,0.66666667,0.24235294,0.25251509]
,[0.36825667,0.15942029,0.48508634,0.66666667,0.27176471,0.2806841]
,[0.39664963,0.1884058,0.52904239,0.66666667,0.29882353,0.31187123]
,[0.42887564,0.20652174,0.57299843,0.66666667,0.32352941,0.34104628]
,[0.45528109,0.24275362,0.58712716,0.66666667,0.35882353,0.36217304]
,[0.48495173,0.26570048,0.59654631,0.66666667,0.39529412,0.37927565]
,[0.50695627,0.29227053,0.60753532,0.66666667,0.43058824,0.40744467]
,[0.5330778,0.32850242,0.62166405,0.66666667,0.45294118,0.42454728]
,[0.5552243,0.34057971,0.64207221,0.66666667,0.49058824,0.44567404]
,[0.57637706,0.36594203,0.67503925,0.66666667,0.50941176,0.4668008,]
,[0.59923339,0.39492754,0.69387755,0.66666667,0.54823529,0.51509054]
,[0.63202726,0.43719807,0.71742543,0.66666667,0.57529412,0.54225352]
,[0.6568711,0.45289855,0.74568289,0.66666667,0.59882353,0.56237425]
,[0.67915957,0.48309179,0.76609105,0.66666667,0.63176471,0.57344064]
,[0.7044293,0.51328502,0.77708006,0.66666667,0.67647059,0.59356137]
,[0.7245883,0.53864734,0.79905808,0.66666667,0.70117647,0.61267606]
,[0.75014196,0.55555556,0.82103611,0.66666667,0.71764706,0.63782696]
,[0.7673197,0.58333333,0.82574568,0.66666667,0.73764706,0.65291751]
,[0.79358319,0.62801932,0.83830455,0.66666667,0.76,0.67102616]
,[0.82027257,0.67270531,0.84929356,0.66666667,0.78352941,0.69919517]
,[0.84085747,0.7089372,0.8744113,0.66666667,0.81294118,0.73038229]
,[0.86697899,0.74516908,0.88540031,0.66666667,0.82823529,0.75653924]
,[0.89040318,0.79589372,0.91522763,0.66666667,0.86588235,0.79476861]
,[0.91993186,0.85024155,0.95290424,0.66666667,0.89411765,0.85412475]
,[0.93796139,0.90700483,0.96546311,0.66666667,0.91529412,0.8943662]
,[0.96919364,0.96497585,0.97645212,0.66666667,0.95294118,0.95171026]
,[1.,1.,1.,1.,1.,1.]])

In [56]:
rand_seed_natural = np.random.rand(1,20)
# 分配结果微调
def create_uniques(arr):
    unq,c = np.unique(arr,return_counts=1)

    m = np.isin(arr,unq[c>1])

    newvals = np.setdiff1d(np.arange(31),arr[~m])
    np.random.shuffle(newvals)
    
    cnt = m.tolist().count(True)
    newvals = newvals[:cnt]
    arr[m] = newvals
    
    return arr

In [59]:
for i in range(20):
    # 将随机数和累积分布挨个对比，概率大的那天区间也大，被选中的概率自然大
    for j in cum_probability_2[:,0]:
        if j >=  rand_seed_natural[0,i]: 
            day_select_natural[0,i] = cum_probability_2[:,0].tolist().index(j)+1
            break
# 微调，作用是若有的天被分配大于1次，则将这天重新打散分配，用create_uniques函数
day_select_natural = create_uniques(np.array(day_select_natural[0]).astype('int64'))

In [60]:
day_select_natural

array([27,  0, 24, 14,  6, 28, 26,  5,  8,  4, 23, 18, 22, 17, 20, 15, 16,
       12,  2,  3, 21,  1,  9, 10, 29, 11, 13, 25,  7, 30])

In [64]:
np.random.rand(1,10)

array([[0.43840969, 0.85466365, 0.30925827, 0.790468  , 0.18639407,
        0.60621843, 0.88316462, 0.10326071, 0.82266334, 0.84382056]])

In [87]:
test = np.zeros(31)
test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [81]:
for i in [1,2,3]:
   # print(i)
    test[i] =1
test

array([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [85]:
test[[1,2,3]]=1

In [86]:
test

array([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [88]:
channel_type = ['natural','com','free','jd_strong_ctrl', 'jd_normal_ctrl']

In [90]:
channel_type.index('com')

1

In [98]:
day_cnt_dict = {'natural',[0]*31}

TypeError: unhashable type: 'list'

In [6]:
day_ago_1 =  '2022-07-01' # T-1


now = datetime.datetime.strptime(day_ago_1, '%Y-%m-%d')
day_ago_1 =  datetime.datetime.strptime(day_ago_1, '%Y-%m-%d').strftime('%Y-%m-%d')
day_ago_3 = datetime.datetime.strftime(now - timedelta(days=-3),'%Y-%m-%d')
day_ago_3

'2022-07-04'